![Garanteo logo](garanteo.png)

---

# Table des matières

1. [Initialisation](#1.-Initialisation)
1. [Cleaning](#2.-Cleaning)
1. [Définition des besoins de donnée pour chaque objectif](#3.-D%C3%A9finition-des-besoins-de-donn%C3%A9e-pour-chaque-objectif)
1. [Enrichissement des tables de base](#4.-Enrichissement-des-tables-de-base)
1. [Création des tables pour chaque objectif](#5.-Cr%C3%A9ation-des-tables-pour-chaque-objectif)
    1. [Objectif A](#5.-Objectif-A)
        1. [Initialisation](#5.A-Initialisation)
        1. [Analyse par campagne](#5.A-Analyse-par-campagne)
        1. [Analyse par canal](#5.A-Analyse-par-canal)
        1. [Analyse par partenaire](#5.A-Analyse-par-partenaire)
    1. [Objectif B](#5.-Objectif-B)
    1. [Objectif C](#5.-Objectif-C)
1. [Analyses / Réponses aux questions](#6.-Analyses-/-Réponses-aux-questions)
    1. [Objectif A](#6.-Objectif-A)
    1. [Objectif B](#6.-Objectif-B)
    1. [Objectif C](#6.-Objectif-C)
1. [Extraction des tables pour Power BI](#7.-Extraction-des-tables-pour-Power-BI)

---

# 1. Initialisation

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
pd.options.display.max_rows

In [ ]:
pd.options.display.max_columns

In [ ]:
df_sessions = pd.read_csv("sessions.csv")
df_sessions

In [ ]:
df_campaigns = pd.read_csv("campaigns.csv")
df_campaigns

In [ ]:
df_events = pd.read_csv("events.csv")
df_events

In [ ]:
df_prospects = pd.read_csv("prospects.csv")
df_prospects

# 2. Cleaning

## Table "df_sessions"

**On enlève la colonne d'indice inutile**

In [ ]:
df_sessions = df_sessions.drop(columns='Unnamed: 0')

**On supprime les doublons**

In [ ]:
df_sessions = df_sessions.drop_duplicates().reset_index(drop=True)

**Transformation des colonnes 'session_started_at' et 'session_ended_at' en datetime**

In [ ]:
df_sessions['session_started_at'] = pd.to_datetime(df_sessions['session_started_at'])
df_sessions['session_ended_at'] = pd.to_datetime(df_sessions['session_ended_at'])

## Table "df_campaigns"

**On enlève la colonne d'indice inutile**

In [ ]:
df_campaigns = df_campaigns.drop(columns='Unnamed: 0')

**Suppression des colonnes 'campaign_start_date' et 'campaign_end_date' (lignes toutes identiques)**

In [ ]:
df_campaigns = df_campaigns.drop(columns=['campaign_start_date', 'campaign_end_date'])

## Table "df_events"

**On enlève la colonne d'indice inutile**

In [ ]:
df_events = df_events.drop(columns='Unnamed: 0')

**On supprime les doublons**

In [ ]:
df_events = df_events.drop_duplicates().reset_index(drop=True)

**Transformation de la colonne 'event_timestamp' en datetime**

In [ ]:
df_events['event_timestamp'] = pd.to_datetime(df_events['event_timestamp'])

**On supprime les NaN de la colonne 'user_id', dont les lignes sont inexploitables**

In [ ]:
df_events = df_events.dropna(subset=['user_id'])

## Table df_prospects

**On enlève la colonne d'indice inutile**

In [ ]:
df_prospects = df_prospects.drop(columns='Unnamed: 0')

**On supprime les doublons**

In [ ]:
df_prospects = df_prospects.drop_duplicates().reset_index(drop=True)

**On convertit la colonne 'prospect_creation_date' en datetime**

In [ ]:
df_prospects['prospect_creation_date'] = pd.to_datetime(df_prospects['prospect_creation_date'])

# 3. Définition des besoins de donnée pour chaque objectif

## Rappel des objectifs

- **Objectif A: Optimisation du Budget d'Acquisition**
    - **But:** Allouer le budget de manière à maximiser le ROI en ciblant les canaux et campagnes les plus performants.
    - **Action:** Analyser les performances historiques des différents canaux d'acquisition et répartir le budget selon leur efficacité prouvée et leur potentiel de croissance.
- **Objectif B: Analyse de Segmentation des Visiteurs**
    - **But:** Comprendre les profils des visiteurs du site de Garanteo pour personnaliser les offres et améliorer l'efficacité de la communication.
    - **Action:** Utiliser les données analytiques du site pour segmenter les visiteurs en groupes homogènes, basés sur leur comportement et leurs préférences.
- **Objectif C: Scoring des Leads via WCB (Web Call Backs)**
    - **But:** Améliorer la conversion en qualifiant et en priorisant les leads générés, pour une gestion plus efficace par l'équipe de vente.
    - **Action:** Mettre en place un modèle de scoring basé sur des critères tels que le comportement sur le site et les informations fournies lors de la demande de rappel, pour évaluer le potentiel de conversion de chaque lead.

## Objectif A: Analyser les performances historiques des différents canaux d'acquisition et répartir le budget selon leur efficacité prouvée et leur potentiel de croissance

**Compréhension du sujet :**

Le site a attiré l'essentiel de ses utilisateurs via des campagnes marketing, sur différents canaux et avec différents partenaires.

Le marketing funnel est le suivant : sessions (22973) > users (6720) > users CTC (2157) > prospects (2060) > clients (180).

Pour évaluer l'efficacité du marketing, on peut considérer différents types de sessions :
- C'est la première fois que le client se connecte sur le site
- La session aboutit sur un CTC
- C'est une visite qui n'est ni la 1ère, ni un CTC

Pour chacune de ces sessions, on peut mesurer la durée et le nombre de clics.

**Besoins de donnée :**

Par campagne :
- Type campagne
- Nombre de sessions
- Nombre de 1ères sessions
- Proportion de 1ères sessions
- Nombre de CTC
- Proportion de CTC
- Nombre de prospects créés par ces CTC et convertis en clients
- Temps cumulé de session
- Temps moyen de session
- Nombre de clics
- Nombre moyen de clics
- Coût
- Coût par clic
- Coût par session
- Coût par minute de session
- Coût par CTC
- Coût par client
- Données par page

Par canal :
- Nombre de sessions
- Nombre de 1ères sessions
- Proportion de 1ères sessions
- Nombre de CTC
- Proportion de CTC
- Nombre de prospects créés par ce CTC et convertis en clients
- Temps cumulé de session
- Temps moyen de session
- Nombre de clics
- Nombre moyen de clics
- Données par page

Par partenaire :
- Nombre de sessions
- Nombre de 1ères sessions
- Proportion de 1ères sessions
- Nombre de CTC
- Proportion de CTC
- Nombre de prospects créés par ce CTC et convertis en clients
- Temps cumulé de session
- Temps moyen de session
- Nombre de clics
- Nombre moyen de clics
- Données par page

**Annexes - Dimensionnement du marketing funnel**

In [ ]:
# sessions
df_sessions["user_id"].notna().sum()

In [ ]:
# active users
df_events["user_id"].nunique()

In [ ]:
# users CTC
df_events[df_events['event_type']=='CTC']["user_id"].nunique()

In [ ]:
df_prospects.shape[0]

In [ ]:
df_prospects[df_prospects['is_client']==1].shape[0]

## Objectif B: Utiliser les données analytiques du site pour segmenter les visiteurs en groupes homogènes, basés sur leur comportement et leurs préférences

**Compréhension du sujet :**

Les utilisateurs visitent des pages différentes, se connectent sur des durées plus ou moins longues, sur des jours de la semaine différents, avec des set-up (device, browser, etc.) différents.

L'idée est de créer un dataframe avec le plus de données comportementales possibles sur tous les utilisateurs (pas seulement les prospects)...

... Puis de faire tourner un algorithme de ML (clustering) pour dégager de potentiels segments clients.

Si le clustering n'est pas concluant, on peut réfléchir à un arbre de décision pour classer les clients selon leurs préférences et leurs habitudes de connexion

**Besoins de donnée :**

Par utilisateur actif (prospect & non-prospect = 6124 utilisateurs) :
- Nombre de sessions
- Temps moyen entre chaque session
- Nombre de visites de chaque page
- Temps total sur le site
- Temps total sur chaque page
- Proportion du temps par page
- Nombre de CTC par page
- Temps de visite par jour de la semaine
- Temps de visite par période de la journée
- Device
- Browser
- OS
- Langue
- Pays

**Annexe - Dimensionnement des utilisateurs**

In [ ]:
df_sessions["user_id"].nunique()

## Objectif C: Mettre en place un modèle de scoring basé sur des critères tels que le comportement sur le site et les informations fournies lors de la demande de rappel, pour évaluer le potentiel de conversion de chaque lead

**Compréhension du sujet :**

- On dispose d'une table de 2060 prospects, 

- 655 ont été rappelés (Groupe 1), et parmi ceux-ci :
    - 47 sont devenus clients
    - 608 ne sont pas devenus clients

- 1272 n'ont toujours pas été rappelés ET ne sont pas clients (Groupe 2). 

C'est le Groupe 2 qui nous intéresse. On cherche à savoir quels prospects de ce groupe, une fois appelés, ont le plus de chance de devenir client.

Pour cela, on va enrichir la base de données Prospects avec des points de données sur le comportement des utilisateurs.

Puis on va entraîner un modèle de ML (catégorisation) sur le Groupe 1, avec y = 'is_client' et X = les données utilisateurs jugées pertinentes.

On fera ensuite tourner le modèle sur le Groupe 2 pour estimer leur probabilité de devenir client.

Discussion sur l'inclusion ou non des clients non-appelés :

https://chatgpt.com/share/66f3cb5b-bdc4-8013-9d06-5cf91c21648a

**Besoins de donnée :**

Variable à prédire (y)
- Is_client (y)

Variables pour l'entraînement (X) :
- Age (et/ou catégorie d'âge)
- Genre
- Nombre de sessions
- Temps total sur le site
- Temps moyen (en jours) entre deux sessions
- Nombre total de clics
- Nombre de visites avant le CTC
- Temps (calendaire) entre la 1ère visite et le 1er CTC
- Temps total en ligne avant le 1er CTC
- Page du 1er et du dernier CTC

Pour les besoins de l'Objectif A :
- Id de la campagne du 1er CTC

Pour prioriser les lead calls une fois les scores assignés :
- Date du dernier CTC

Ensuite on filtre le Groupe 1 et on entraîne le modèle de ML

Puis on filtre le Groupe 2, on vide la colonne is_client, et on fait tourner le modèle 


**Annexe - Dimensionnement des échantillons**

In [ ]:
# Groupe 1
df_prospects[df_prospects['is_presented_prospect']==1]['is_client'].value_counts()

In [ ]:
# Groupe 2
df_prospects[(df_prospects['is_presented_prospect']==0) & (df_prospects['is_client']==0)].shape[0]

# 4. Enrichissement des tables de base

## Table df_sessions

**On ajoute la durée de chaque session**

In [ ]:
df_sessions['session_duration'] = (df_sessions['session_ended_at'] - df_sessions['session_started_at'])

**On ajoute le jour de la semaine de la session**

In [ ]:
df_sessions['session_weekday'] = df_sessions['session_started_at'].dt.weekday

**On crée des "moments de la journée"**
- 00h00 : 06h00 = "night"
- 06h00 : 12h00 = "morning"
- 12h00 : 18h00 = "afternoon"
- 18h00 : 00h00 = "evening"

In [ ]:
def day_period(time):
    if (time >= dt.time(0,0,0)) & (time < dt.time(6,0,0)):
        return 'night'
    elif (time >= dt.time(6,0,0)) & (time < dt.time(12,0,0)):
        return 'morning'
    elif (time >= dt.time(12,0,0)) & (time < dt.time(18,0,0)):
        return 'afternoon'
    elif (time >= dt.time(18,0,0)) & (time <= dt.time(23,59,59)):
        return 'evening'
    else:
        return np.nan

In [ ]:
df_sessions['day_period'] = df_sessions['session_started_at'].dt.time.apply(day_period)

**Ajout du temps entre deux sessions d'un même utilisateur**

In [ ]:
# On trie les sessions par utilisateur et par date/heure de début de session
df_sessions = df_sessions.sort_values(['user_id','session_started_at']).reset_index(drop=True)

# On ne garde que les indicateurs nécessaires
df_time_btw_sessions = df_sessions[['session_id','user_id', 'session_started_at', 'session_ended_at']]

# Pour chaque ligne, on ajoute la date/heure de la fin de la session juste au-dessus dans le tableau
df_time_btw_sessions['previous_session_end'] = df_time_btw_sessions['session_ended_at'].shift(1)

# Idem, avec l'id utilisateur (on veut vérifier si deux sessions consécutives sont du même utilisateur ou non)
df_time_btw_sessions['previous_user_id'] = df_time_btw_sessions['user_id'].shift(1)

# On crée une colonne qui contient des True quand deux sessions consécutives sont du même user, et NaN sinon
df_time_btw_sessions['is_same_user'] = (df_time_btw_sessions['previous_user_id'] == df_time_btw_sessions['user_id']).replace(False,np.nan)

# On multiplie cette dernière colonne par la la différence entre date/heure de début de la session et date/heure de fin de la précédente
df_time_btw_sessions['time_btw_sessions'] = df_time_btw_sessions['is_same_user']*(df_time_btw_sessions['session_started_at'] - df_time_btw_sessions['previous_session_end'])

# Ainsi, on obtient NaN si deux sessions ne sont pas consécutives (cad, comme la table est triée, si c'est la 1ere session du user)

In [ ]:
# On vérifie qu'il n'y a pas de doublon
df_time_btw_sessions[['session_id', 'session_started_at']].duplicated().sum()

In [ ]:
# On ajoute cette nouvelle colonne à sessions
df_sessions = df_sessions\
                .merge(df_time_btw_sessions[["session_id", "user_id", "session_started_at","time_btw_sessions"]],\
                       on=["session_id", "user_id", "session_started_at"])

**On prépare, pour plus tard, une table avec le temps moyen par session par user**

In [ ]:
df_avgtime_btw_sessions = df_time_btw_sessions.groupby('user_id').agg({'time_btw_sessions':'mean'}).reset_index()
df_avgtime_btw_sessions = df_avgtime_btw_sessions.rename(columns={'time_btw_sessions':'avg_time_btw_sessions'})

**Pour les users dont device_type == desktop, on remplace le NaN de device_operating_system par Windows**

Cela va notamment nous permettre de les distinguer des vrais NaN (absence de données)

In [ ]:
windows = (df_sessions['device_type'] == 'desktop').replace({False: np.nan, True: 'windows'})
df_sessions['device_operating_system'].fillna(windows, inplace=True)

## Table df_events

**Création d'une colonne prélevant les noms de site des url de la colonne 'referrer'**

In [ ]:
def f_referrer(string):
    string = str(string)
    if 'site' in string:
        return 'site'
    elif 'google' in string:
        return 'google'
    elif 'awin' in string:
        return 'awin'
    elif 'affise' in string:
        return 'affise'
    elif 'cj' in string:
        return 'cj'
    else:
        return np.nan

In [ ]:
df_events['referrer_summary'] = df_events['referrer'].apply(f_referrer)

**Création d'une colonne avec les numéros de page visitées (pour simplifier la colonne url)**

In [ ]:
df_events['page'] = df_events['url'].apply(lambda x: int(x[-1])).astype(str)

## Table df_prospects

**On crée des catégories d'âge**

In [ ]:
def age_cat_w10(x):
    for i in range(20, 200, 10):
        if (x>i-10) & (x<=i):
            age_cat = f'{i-10+1}-{i}'
    return age_cat

def age_cat_w20(x):
    for i in range(20, 200, 20):
        if (x>i-20) & (x<=i):
            age_cat = f'{i-20+1}-{i}'
    return age_cat

def age_cat_w30(x):
    for i in range(20, 200, 30):
        if (x>i-30) & (x<=i):
            age_cat = f'{i-30+1}-{i}'
    return age_cat

def age_cat_w40(x):
    for i in range(20, 200, 40):
        if (x>i-40) & (x<=i):
            age_cat = f'{i-40+1}-{i}'
    return age_cat

In [ ]:
# On ne crée que les colonnes de largeur 20y et 30y
df_prospects['age_cat_w20'] = df_prospects['age'].apply(age_cat_w20)
df_prospects['age_cat_w30'] = df_prospects['age'].apply(age_cat_w30)

# 5. Création des tables pour chaque objectif

## 5. Objectif A

### 5.A Initialisation

**Jointure df_sessions / df_events et enrichissement de cette table**

On rappelle qu'en supprimant les user_id = nan dans df_events, on a aussi supprimé les session_id = nan

On veut maintenant associer chaque event à une session 

Comme il y a des session_id = nan dans df_sessions, je veux vérifier, au préalable, qu'en me focalisant sur les session_id non-nan, je ne perds pas un volume de données significatifs sur les users qui sont dans la table events

In [ ]:
df_sessions[df_sessions['user_id'].isin(df_events['user_id'])]['session_id'].isna().sum()

Je vais perdre 1 ligne, c'est acceptable. Je peux donc effectuer ma jointure :

In [ ]:
df_left = df_sessions
df_right = df_events[['event_id','session_id','event_timestamp','event_type','page','referrer_summary','medium','campaign_id','user_id']]
df_analysis = pd.merge(left = df_left, right = df_right, how = 'left', on = ['session_id', 'user_id'])\
                .sort_values(['user_id', 'event_timestamp'])\
                .reset_index(drop=True)

**On compte les clicks & CTC par session**

In [ ]:
df_session_clickcount = df_analysis[['session_id','event_type','user_id']]\
                            .groupby(['session_id','event_type'])\
                            .agg({'user_id':'count'})\
                            .reset_index()\
                            .rename(columns={'user_id':'event_count'})

df_session_clickcount = df_session_clickcount.sort_values('session_id')

# On veut 1 colonne click et 1 colonne CTC, on utilise donc .pivot()
df_session_clickcount = df_session_clickcount.pivot(columns='event_type', index='session_id', values='event_count').reset_index()

In [ ]:
df_analysis = df_analysis.merge(df_session_clickcount, how='left', on='session_id')

**On crée une colonne où, pour chaque event, toutes les lignes renvoient le timestamp du CTC qui a eu lieu dans la même session, s'il y en a eu un**

In [ ]:
df_ctc_timestamp = df_analysis[df_analysis['event_type']=='CTC'][['session_id', 'user_id', 'event_timestamp']]\
                        .rename(columns={'event_timestamp':'CTC_timestamp'})

df_analysis = df_analysis.merge(df_ctc_timestamp, how='left', on=['session_id', 'user_id'])

**On crée un boolean qui indique si la session de chaque événement est la 1ère de l'utilisateur ou non**

In [ ]:
df_first_session = df_sessions[['user_id','session_started_at']]\
                        .sort_values(['user_id', 'session_started_at'])\
                        .groupby('user_id')\
                        .agg({'session_started_at':'min'}).reset_index()\
                        .rename(columns={'session_started_at':'first_session_started_at'})

df_analysis = df_analysis.merge(df_first_session, how='left', on='user_id')

df_analysis['is_first_session'] = (df_analysis['session_started_at'] == df_analysis['first_session_started_at'])

**Q =** Toute session commence-t-elle par un event ?

In [ ]:
df_start = df_analysis[df_analysis['session_started_at'] == df_analysis['event_timestamp']]
df_start.shape[0]

In [ ]:
(df_sessions['session_id'].notna()).sum()

**R =** La *plupart* des sessions commencent par un event

**Q =** Quid de celles qui ne commencent pas par un event ?

In [ ]:
df_weird_sessions = df_sessions[(~df_sessions['session_id'].isin(df_start['session_id'])) & (df_sessions['session_id'].notna())]
df_weird_sessions.shape[0]

On retrouve nos 20514 - 20266 = 248 sessions qui ne commencent pas par un event

On reprend notre df_analysis mais on filtre les session_id, pour avoir les calculs effectués précédemment, appliqués à ces sessions-là

In [ ]:
df_events[df_events['session_id'].isin(df_weird_sessions['session_id'])]

**R =** Ces sessions ne commencent pas par un événement, pour la simple et bonne raison qu'*il n'y a pas d'événement associé à ces sessions dans df_events*

Ce sont donc des sessions d'utilisateurs enregistrés dans df_events, mais dont les sessions n'ont généré aucun événement

**CONCLUSION** : en dehors de 248 sessions qui n'ont généré aucun événement, les 20266 autres sessions commencent toutes par un événement

**Création d'une sous-table centrée sur les utilisateurs ayant fait un CTC**

In [ ]:
# Je me limite aux sessions qui aboutissent sur un CTC
df_ctc_users = df_analysis[df_analysis['CTC']>0][['session_id','user_id','CTC_timestamp']]

# Pour l'instant, pour chaque session, j'ai autant de lignes que d'événements
# Je n'en veux qu'une seule : 
    # Pour le tout 1er CTC de l'utilisateur : le 1er CTC de chaque sessions (.agg(min))
    # Pour celle du 1er CTC de la session : le dernier CTC de chaque sessions (.agg(max))

**Premier CTC de chaque utilisateur**

In [ ]:
# 1er CTC de chaque session :
df_firstctc = df_ctc_users.groupby(['session_id','user_id']).agg({'CTC_timestamp':'min'}).reset_index()

# On trie par date, puis on supprime les doublons pour ne garder que le 1er CTC de chaque utilisateur
df_firstctc = df_firstctc.sort_values(['user_id','CTC_timestamp']).drop_duplicates('user_id')

# On rajoute la date/heure du dédut de session et les infos marketing liées à cette session
df_firstctc = df_firstctc.merge(df_analysis[['session_id', 'event_timestamp', 'campaign_id', 'medium', 'referrer_summary']]\
                                   .sort_values(['session_id', 'event_timestamp'])\
                                   .drop_duplicates('session_id'),\
                                how='left', on='session_id')\
                        .rename(columns = {'event_timestamp':'session_started_at'})

**On ajoute ensuite les données de conversion : en prospect, en client** 

In [ ]:
df_firstctc = df_firstctc.merge(df_prospects[['user_id', 'prospect_creation_date', 'is_client']], how='left', on='user_id')

Sur 2157 users distincts ayant fait un CTC, 2060 sont devenus des prospects

**Analyse par page**

On veut calculer le nb de visites et le temps passé sur chaque page par utilisateur

Il faut pour cela regarder la durée de chaque événement

Deux méthodes de calcul :
- *Si l'événement **n'est pas** le dernier de sa session* : event_timestamp de l'événement juste après dans la même session *moins* event_timestamp de l'événement considéré 
- *Si l'événement **est** le dernier de sa session* : session_ended_at *moins* event_timestamp de l'événement considéré

In [ ]:
# On extrait les données requises de table analysis (sessions left join events)
df_page_visits = df_analysis[['user_id', 'event_timestamp', 'session_started_at', 'session_ended_at', 'event_type', 'page']]

In [ ]:
# On trie par user_id et par date/heure d'événement
df_page_visits = df_page_visits.sort_values(['user_id', 'event_timestamp'])

# On crée une colonne avec la date/heure de l'événement juste après
df_page_visits['next_event'] = df_page_visits['event_timestamp'].shift(-1)

# Quand je tombe sur un NaN dans event_timestamp (session sans événement),
# event_timestamp prend la valeur de session_started_at
df_page_visits['event_timestamp'] = df_page_visits['event_timestamp'].fillna(df_page_visits['session_started_at'])

# Quand je tombe sur un NaN dans next_event (le prochain event est une session sans event),
# next_event prend la valeur de event_timestamp
df_page_visits['next_event'] = df_page_visits['next_event'].fillna(df_page_visits['event_timestamp'])

In [ ]:
# On extrait le dernier événement de chaque session grâce à un .groupby.agg(max)
df_last_event = df_page_visits[['user_id', 'session_started_at', 'event_timestamp']]\
                    .groupby(['user_id', 'session_started_at'])\
                    .agg({'event_timestamp':'max'})\
                    .reset_index()\
                    .rename(columns={'event_timestamp':'last_event_timestamp'})

# On l'ajoute à notre table précédemment créée
df_page_visits = df_page_visits.merge(df_last_event, on=['user_id', 'session_started_at'])

# On crée une colonne indiquant si chaque événement est le dernier de sa session ou non
df_page_visits['is_last_event'] = (df_page_visits['last_event_timestamp'] == df_page_visits['event_timestamp'])

In [ ]:
# Si l'événement N'EST PAS le dernier de sa session, c'est la ligne 2 qui s'active
# La durée de l'événement est alors égale à next_event moins event_timestamp
# Si au contraire l'événement EST le dernier de sa session, c'est la ligne 3 qui s'active
# La durée de l'événement est alors égale à session_ended_at moins event_timestamp
df_page_visits['event_duration_in_sec'] =\
    (df_page_visits['next_event'] - df_page_visits['event_timestamp'])*(~df_page_visits['is_last_event']) +\
    (df_page_visits['session_ended_at'] - df_page_visits['event_timestamp'])*(df_page_visits['is_last_event'])

# Si on n'avait pas traité les NaN précédemment, on aurait eu des erreurs
# Les sessions sans events activent la ligne 3, avec event_timestamp = session_started_at,
# donc la différence devient égale à la durée de la session

In [ ]:
# On convertit les event_duration en secondes (au lieu de timedelta)
df_page_visits['event_duration_in_sec'] = df_page_visits['event_duration_in_sec'].dt.total_seconds()

# On extrait la colonne event_duration
df_event_duration = df_page_visits[['user_id', 'event_timestamp', 'event_duration_in_sec']]

# On rajoute une colonne pour savoir si l'événement est un CTC
df_page_visits['is_CTC'] = (df_page_visits['event_type']=='CTC')

# On crée une copie de df_page_visits. 
# df_sessions_breakdown nous servira plus tard à l'analyse par campagne / par medium / par partenaire
# df_page_visits sera transformé dans l'Objectif B pour l'analyse de visites de page par user

df_sessions_breakdown = df_page_visits

### 5.A Analyse par campagne

**On vérifie qu'on peut faire un groupby par session_id**

(en regardant si les session_id = NaN peuvent quand même avoir un campaign_id)

In [ ]:
df_analysis[df_analysis['session_id'].isna()]['campaign_id'].value_counts()

Réponse : non, donc on peut groupby par session_id

**On vérifie ensuite qu'à chaque session est associée une unique campagne**

In [ ]:
df_analysis[['session_id', 'campaign_id']].groupby('session_id').value_counts()

Réponse : oui

**Pour chaque campagne, on commence par calculer le nombre de clics, le nombre de sessions, le temps en ligne**

In [ ]:
# On extrait les colonnes qui nous intéressent
df_sessions_campaigns = df_analysis[['campaign_id', 'session_id', 'session_duration', 'is_first_session', 'click', 'CTC']]

# On ne garde qu'une ligne par session, puisqu'on a déjà des colonnes qui comptent les clicks / CTC / durée de session
df_sessions_campaigns = df_sessions_campaigns.drop_duplicates(subset='session_id')

print(f"Nombre de campaign_id = NaN (et donc autant de sessions perdues): {df_sessions_campaigns['campaign_id'].isna().sum()}")

# On remplace les NaN par zéro
df_sessions_campaigns[['click', 'CTC']] = df_sessions_campaigns[['click', 'CTC']].fillna(0)

# On procède à un groupby + agg
dico_agg = {
    'session_id':'count',
    'session_duration':['sum', 'mean'],
    'is_first_session':['sum','mean'],
    'click':['sum', 'mean'],
    'CTC':['sum', 'mean']
}

df_sessions_campaigns = df_sessions_campaigns.groupby('campaign_id')\
                            .agg(dico_agg)\
                            .reset_index()

# On traite chaque bloc du tableau pour supprimer le niveau 0 et renommer les colonnes de la bonne façon
df_sessions_count = df_sessions_campaigns[['campaign_id', 'session_id']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'campaign_id', 'count':'session_count'})

df_sessions_duration = df_sessions_campaigns[['campaign_id', 'session_duration']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'campaign_id', 'sum':'sum_duration_in_min', 'mean':'mean_duration_in_min'})

df_sessions_first = df_sessions_campaigns[['campaign_id', 'is_first_session']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'campaign_id', 'sum':'sum_first_session', 'mean':'prop_first_session'})

df_sessions_click = df_sessions_campaigns[['campaign_id', 'click']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'campaign_id', 'sum':'sum_click', 'mean':'mean_click'})

df_sessions_ctc = df_sessions_campaigns[['campaign_id', 'CTC']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'campaign_id', 'sum':'sum_ctc', 'mean':'prop_ctc'})

# On fusionne tout
df_sessions_campaigns = df_sessions_count\
                            .merge(df_sessions_duration, on='campaign_id')\
                            .merge(df_sessions_first, on='campaign_id')\
                            .merge(df_sessions_click, on='campaign_id')\
                            .merge(df_sessions_ctc, on='campaign_id')\
                            .merge(df_campaigns, on='campaign_id')

# On arrondit la durée totale en minutes
df_sessions_campaigns['sum_duration_in_min'] = round(\
                                                df_sessions_campaigns['sum_duration_in_min'].dt.days*24*60 +\
                                                df_sessions_campaigns['sum_duration_in_min'].dt.seconds/60\
                                                    ).astype(int)
df_sessions_campaigns['mean_duration_in_min'] = round(\
                                                    df_sessions_campaigns['mean_duration_in_min'].dt.seconds/60\
                                                        ).astype(int)

# On calcule des ratios de coûts à différentes étapes du marketing funnel
df_sessions_campaigns['cost_per_session'] = df_sessions_campaigns['total_cost'] / df_sessions_campaigns['session_count']

df_sessions_campaigns['cost_per_click'] = df_sessions_campaigns['total_cost'] / df_sessions_campaigns['sum_click']

df_sessions_campaigns['cost_per_min'] = df_sessions_campaigns['total_cost'] / df_sessions_campaigns['sum_duration_in_min']

df_sessions_campaigns['cost_per_ctc'] = df_sessions_campaigns['total_cost'] / df_sessions_campaigns['sum_ctc']

**On reprend la table des 1ers CTCs, et on groupe par campagne, pour calculer leurs nombres de prospects créés CTC puis convertis en clients**

In [ ]:
dico_agg = {
    'user_id':'count',
    'prospect_creation_date':'count',
    'is_client':'sum'
}

dico_rename = {
    'user_id':'users_on_CTC',
    'prospect_creation_date':'prospect_created',
    'is_client':'client_acquired'
}

df_ctc_campaign = df_firstctc[['campaign_id', 'user_id', 'prospect_creation_date', 'is_client']]\
                        .groupby('campaign_id')\
                        .agg(dico_agg)\
                        .reset_index()\
                        .rename(columns = dico_rename)

In [ ]:
df_ctc_campaign['prospect_created'].sum() + df_firstctc['campaign_id'].isna().sum()

**Pour chaque campagne, on compte les redirections vers chaque page (au début des sessions)**

In [ ]:
df_campaigns_redirect = df_analysis[df_analysis['campaign_id'].notna()][['campaign_id','page']].value_counts().to_frame()\
                            .sort_values(['campaign_id', 'page']).reset_index()\
                            .pivot(index='campaign_id', columns='page', values='count').reset_index()\
                            .rename(columns={
                            '1':'redirect_p1',
                            '2':'redirect_p2',
                            '3':'redirect_p3',
                            '4':'redirect_p4',
                            '5':'redirect_p5',
                            '6':'redirect_p6'
                        })

**On calcule le temps sur chaque page (en absolu et en %), le nombre de CTC par page (en absolu et en %)**

In [ ]:
# Tableau pour afficher des graphes par page en utilisant le paramètre "hue"

# On extrait les colonnes requises du tableau df_sessions_breakdown
df_campaign_pagefocus = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']]

# groupby: on groupe par session et par page
# agg : Pour chaque session, on calcule la somme du temps passé par page, et le nombre de CTC par page 
df_campaign_pagefocus = df_campaign_pagefocus.groupby(['user_id', 'session_started_at', 'page'])\
                            .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                            .reset_index()

# Pour chacune de ces sessions, on ajoute la campagne qui a initié la session 
df_campaign_pagefocus = df_campaign_pagefocus.merge(df_analysis[['user_id', 'session_started_at', 'campaign_id']]\
                                                        .dropna(subset='campaign_id'),\
                                                    on=['user_id', 'session_started_at'])

# drop: on se débarasse des références à la session 
# groupby : on groupe une nouvelle fois par campagne et par page
# agg : pour chacune des campagnes, on calcule le temps passé par les utilisateurs sur chaque page, et le nb de CTC par page
df_campaign_pagefocus = df_campaign_pagefocus.drop(columns=['user_id', 'session_started_at'])\
                            .groupby(['campaign_id', 'page'])\
                            .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                            .reset_index()\
                            .rename(columns={'event_duration_in_sec':'cumul_time_in_sec', 'is_CTC':'CTC_count'})

In [ ]:
# Tableau pivoté
# Même principe au début
df_campaign_pagefocus_pivot = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']]\
                                .groupby(['user_id', 'session_started_at', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()\
                                .merge(df_analysis[['user_id', 'session_started_at', 'campaign_id']].dropna(subset='campaign_id'),\
                                       on=['user_id', 'session_started_at'])\
                                .drop(columns=['user_id', 'session_started_at'])\
                                .groupby(['campaign_id', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()

# On rajoute un pivot pour avoir une colonne par page, pour chaque indicateur
df_campaign_pagefocus_pivot = df_campaign_pagefocus_pivot.pivot(index='campaign_id', columns='page', values=['event_duration_in_sec', 'is_CTC'])\
                                .reset_index()\
# On cleane ensuite par bloc :
df_campaign_pagetime = df_campaign_pagefocus_pivot['event_duration_in_sec']\
                        .rename(columns={
                            '1':'cumul_time_p1',
                            '2':'cumul_time_p2',
                            '3':'cumul_time_p3',
                            '4':'cumul_time_p4',
                            '5':'cumul_time_p5',
                            '6':'cumul_time_p6'
                        })

df_campaign_pagectc = df_campaign_pagefocus_pivot['is_CTC']\
                        .rename(columns={
                            '1':'CTC_count_p1',
                            '2':'CTC_count_p2',
                            '3':'CTC_count_p3',
                            '4':'CTC_count_p4',
                            '5':'CTC_count_p5',
                            '6':'CTC_count_p6'
                        })

# On concatène le tout
df_campaign_pagefocus_pivot = pd.concat([
                                df_campaign_pagefocus_pivot['campaign_id'], 
                                df_campaign_pagetime, 
                                df_campaign_pagectc
                                ],axis=1)

In [ ]:
# On calcule des sommes pour ensuite calculer des %
df_campaign_pagefocus_pivot['cumul_time_all']=\
    df_campaign_pagefocus_pivot[['cumul_time_p1','cumul_time_p2','cumul_time_p3','cumul_time_p4','cumul_time_p5','cumul_time_p6']]\
    .sum(axis=1)

df_campaign_pagefocus_pivot['CTC_count_all']=\
    df_campaign_pagefocus_pivot[['CTC_count_p1','CTC_count_p2','CTC_count_p3','CTC_count_p4','CTC_count_p5','CTC_count_p6']]\
    .sum(axis=1)

In [ ]:
# Calcul des % par page pour les 2 indicateurs
df_campaign_pagefocus_pivot['prop_time_p1']= df_campaign_pagefocus_pivot['cumul_time_p1'] / df_campaign_pagefocus_pivot['cumul_time_all']
df_campaign_pagefocus_pivot['prop_time_p2']= df_campaign_pagefocus_pivot['cumul_time_p2'] / df_campaign_pagefocus_pivot['cumul_time_all']
df_campaign_pagefocus_pivot['prop_time_p3']= df_campaign_pagefocus_pivot['cumul_time_p3'] / df_campaign_pagefocus_pivot['cumul_time_all']
df_campaign_pagefocus_pivot['prop_time_p4']= df_campaign_pagefocus_pivot['cumul_time_p4'] / df_campaign_pagefocus_pivot['cumul_time_all']
df_campaign_pagefocus_pivot['prop_time_p5']= df_campaign_pagefocus_pivot['cumul_time_p5'] / df_campaign_pagefocus_pivot['cumul_time_all']
df_campaign_pagefocus_pivot['prop_time_p6']= df_campaign_pagefocus_pivot['cumul_time_p6'] / df_campaign_pagefocus_pivot['cumul_time_all']

df_campaign_pagefocus_pivot['prop_CTC_p1']= df_campaign_pagefocus_pivot['CTC_count_p1'] / df_campaign_pagefocus_pivot['CTC_count_all']
df_campaign_pagefocus_pivot['prop_CTC_p2']= df_campaign_pagefocus_pivot['CTC_count_p2'] / df_campaign_pagefocus_pivot['CTC_count_all']
df_campaign_pagefocus_pivot['prop_CTC_p3']= df_campaign_pagefocus_pivot['CTC_count_p3'] / df_campaign_pagefocus_pivot['CTC_count_all']
df_campaign_pagefocus_pivot['prop_CTC_p4']= df_campaign_pagefocus_pivot['CTC_count_p4'] / df_campaign_pagefocus_pivot['CTC_count_all']
df_campaign_pagefocus_pivot['prop_CTC_p5']= df_campaign_pagefocus_pivot['CTC_count_p5'] / df_campaign_pagefocus_pivot['CTC_count_all']
df_campaign_pagefocus_pivot['prop_CTC_p6']= df_campaign_pagefocus_pivot['CTC_count_p6'] / df_campaign_pagefocus_pivot['CTC_count_all']

**Méga-jointure**

In [ ]:
df_campaigns_metrics = pd.merge(left = df_sessions_campaigns, right = df_ctc_campaign, on = 'campaign_id')\
                            .merge(df_campaigns_redirect, on='campaign_id')\
                            .merge(df_campaign_pagefocus_pivot, on='campaign_id').drop(columns=['CTC_count_all', 'cumul_time_all'])

**Calcul du coût par prospect et par client**

In [ ]:
df_campaigns_metrics['cost_per_prospect'] = df_campaigns_metrics['total_cost'] / df_campaigns_metrics['prospect_created']
df_campaigns_metrics['cost_per_client'] = df_campaigns_metrics['total_cost'] / df_campaigns_metrics['client_acquired']

**Tableau final**

In [ ]:
df_campaigns_metrics

### 5.A Analyse par canal

**Pour chaque canal (medium), on commence par calculer le nombre de clics, le nombre de sessions, le temps en ligne**

In [ ]:
# On extrait les colonnes qui nous intéressent
df_sessions_medium = df_analysis[['medium', 'session_id', 'session_duration', 'is_first_session', 'click', 'CTC']]

# On ne garde qu'une ligne par session, puisqu'on a déjà des colonnes qui comptent les clicks / CTC / durée de session
df_sessions_medium = df_sessions_medium.drop_duplicates(subset='session_id')

# On remplace les NaN par zéro
df_sessions_medium[['click', 'CTC']] = df_sessions_medium[['click', 'CTC']].fillna(0)

# On procède à un groupby + agg
dico_agg = {
    'session_id':'count',
    'session_duration':['sum', 'mean'],
    'is_first_session':['sum', 'mean'],
    'click':['sum', 'mean'],
    'CTC':['sum', 'mean']
}

df_sessions_medium = df_sessions_medium.groupby('medium')\
                            .agg(dico_agg)

df_sessions_medium = df_sessions_medium.reset_index()

# On traite chaque bloc du tableau pour supprimer le niveau 0 et renommer les colonnes de la bonne façon
df_sessions_count = df_sessions_medium[['medium', 'session_id']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'medium', 'count':'session_count'})

df_sessions_duration = df_sessions_medium[['medium', 'session_duration']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'medium', 'sum':'sum_duration_in_min', 'mean':'mean_duration_in_min'})

df_sessions_first = df_sessions_medium[['medium', 'is_first_session']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'medium', 'sum':'sum_first_session', 'mean':'prop_first_session'})

df_sessions_click = df_sessions_medium[['medium', 'click']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'medium', 'sum':'sum_click', 'mean':'mean_click'})

df_sessions_ctc = df_sessions_medium[['medium', 'CTC']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'medium', 'sum':'sum_ctc', 'mean':'prop_ctc'})

# On fusionne tout
df_sessions_medium = df_sessions_count\
                            .merge(df_sessions_duration, on='medium')\
                            .merge(df_sessions_first, on='medium')\
                            .merge(df_sessions_click, on='medium')\
                            .merge(df_sessions_ctc, on='medium')\

# On arrondit la durée totale en minutes
df_sessions_medium['sum_duration_in_min'] = round(\
                                                df_sessions_medium['sum_duration_in_min'].dt.days*24*60 +\
                                                df_sessions_medium['sum_duration_in_min'].dt.seconds/60\
                                                    ).astype(int)
df_sessions_medium['mean_duration_in_min'] = round(\
                                                    df_sessions_medium['mean_duration_in_min'].dt.seconds/60\
                                                        ).astype(int)

**On reprend la table des 1ers CTCs, et on groupe par medium, pour calculer leurs nombres de prospects créés CTC puis convertis en clients**

In [ ]:
dico_agg = {
    'user_id':'count',
    'prospect_creation_date':'count',
    'is_client':'sum'
}

dico_rename = {
    'user_id':'users_on_CTC',
    'prospect_creation_date':'prospect_created',
    'is_client':'client_acquired'
}

df_ctc_medium = df_firstctc[['medium', 'user_id', 'prospect_creation_date', 'is_client']]\
                        .groupby('medium')\
                        .agg(dico_agg)\
                        .reset_index()\
                        .rename(columns = dico_rename)

In [ ]:
df_ctc_medium['prospect_created'].sum()

**Pour chaque medium, on compte les redirections vers chaque page (au début des sessions)**

In [ ]:
# On commence par ne prendre les événements de df_analysis que lorsque 'medium' n'est pas une redirection interne
# On ne prend que les deux colonnes dont on a besoin
df_medium_redirect = df_analysis[df_analysis['medium'].replace('internal', np.nan).notna()][['medium','page']]

# On fait un value_counts qu'on pivote ensuite pour avoir 6 colonnes qui comptent le nombre de redirections sur les 6 pages
df_medium_redirect = df_medium_redirect.value_counts().to_frame()\
                            .sort_values(['medium', 'page']).reset_index()\
                            .pivot(index='medium', columns='page', values='count').reset_index()\
                            .rename(columns={
                            '1':'redirect_p1',
                            '2':'redirect_p2',
                            '3':'redirect_p3',
                            '4':'redirect_p4',
                            '5':'redirect_p5',
                            '6':'redirect_p6'
                        })

**Calcul du temps par page et du nombre de CTC sur chaque page, en absolu et en relatif pour chaque medium**

In [ ]:
# Tableau pour afficher des graphes par page en utilisant le paramètre "hue"

# On extrait les colonnes requises du tableau df_sessions_breakdown
df_medium_pagefocus = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']]

# groupby: on groupe par session et par page
# agg : Pour chaque session, on calcule la somme du temps passé par page, et le nombre de CTC par page 
df_medium_pagefocus = df_medium_pagefocus.groupby(['user_id', 'session_started_at', 'page'])\
                        .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                        .reset_index()

# Pour chacune de ces sessions, on ajoute le medium qui a initié la session 
df_medium_pagefocus = df_medium_pagefocus.merge(df_analysis[['user_id', 'session_started_at', 'medium']]\
                                                   .replace('internal', np.nan).dropna(subset='medium'),\
                                               on=['user_id', 'session_started_at'])

# drop: on se débarasse des références à la session 
# groupby : on groupe une nouvelle fois par medium et par page
# agg : pour chacun des media, on calcule le temps passé par les utilisateurs sur chaque page, et le nb de CTC par page
df_medium_pagefocus = df_medium_pagefocus.drop(columns=['user_id', 'session_started_at'])\
                        .groupby(['medium', 'page'])\
                        .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                        .reset_index()\
                        .rename(columns={'event_duration_in_sec':'cumul_time_in_sec', 'is_CTC':'CTC_count'})

In [ ]:
# Tableau pivoté
# Même principe au début
df_medium_pagefocus_pivot = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']]\
                                .groupby(['user_id', 'session_started_at', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()\
                                .merge(df_analysis[['user_id', 'session_started_at', 'medium']]\
                                       .replace('internal', np.nan).dropna(subset='medium'),\
                                       on=['user_id', 'session_started_at'])\
                                .drop(columns=['user_id', 'session_started_at'])\
                                .groupby(['medium', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()

# On rajoute un pivot pour avoir une colonne par page, pour chaque indicateur
df_medium_pagefocus_pivot = df_medium_pagefocus_pivot.pivot(index='medium', columns='page', values=['event_duration_in_sec', 'is_CTC'])\
                                .reset_index()

# On cleane ensuite par bloc :
df_medium_pagetime = df_medium_pagefocus_pivot['event_duration_in_sec']\
                        .rename(columns={
                            '1':'cumul_time_p1',
                            '2':'cumul_time_p2',
                            '3':'cumul_time_p3',
                            '4':'cumul_time_p4',
                            '5':'cumul_time_p5',
                            '6':'cumul_time_p6'
                        })

df_medium_pagectc = df_medium_pagefocus_pivot['is_CTC']\
                        .rename(columns={
                            '1':'CTC_count_p1',
                            '2':'CTC_count_p2',
                            '3':'CTC_count_p3',
                            '4':'CTC_count_p4',
                            '5':'CTC_count_p5',
                            '6':'CTC_count_p6'
                        })

# On concatène le tout
df_medium_pagefocus_pivot = pd.concat([
                                df_medium_pagefocus_pivot['medium'], 
                                df_medium_pagetime, 
                                df_medium_pagectc
                                ],axis=1)

In [ ]:
# On calcule des totaux pour ensuite calculer des %
df_medium_pagefocus_pivot['cumul_time_all']=\
    df_medium_pagefocus_pivot[['cumul_time_p1','cumul_time_p2','cumul_time_p3','cumul_time_p4','cumul_time_p5','cumul_time_p6']]\
    .sum(axis=1)

df_medium_pagefocus_pivot['CTC_count_all']=\
    df_medium_pagefocus_pivot[['CTC_count_p1','CTC_count_p2','CTC_count_p3','CTC_count_p4','CTC_count_p5','CTC_count_p6']]\
    .sum(axis=1)

In [ ]:
# Calcul des %
df_medium_pagefocus_pivot['prop_time_p1']= df_medium_pagefocus_pivot['cumul_time_p1'] / df_medium_pagefocus_pivot['cumul_time_all']
df_medium_pagefocus_pivot['prop_time_p2']= df_medium_pagefocus_pivot['cumul_time_p2'] / df_medium_pagefocus_pivot['cumul_time_all']
df_medium_pagefocus_pivot['prop_time_p3']= df_medium_pagefocus_pivot['cumul_time_p3'] / df_medium_pagefocus_pivot['cumul_time_all']
df_medium_pagefocus_pivot['prop_time_p4']= df_medium_pagefocus_pivot['cumul_time_p4'] / df_medium_pagefocus_pivot['cumul_time_all']
df_medium_pagefocus_pivot['prop_time_p5']= df_medium_pagefocus_pivot['cumul_time_p5'] / df_medium_pagefocus_pivot['cumul_time_all']
df_medium_pagefocus_pivot['prop_time_p6']= df_medium_pagefocus_pivot['cumul_time_p6'] / df_medium_pagefocus_pivot['cumul_time_all']

df_medium_pagefocus_pivot['prop_CTC_p1']= df_medium_pagefocus_pivot['CTC_count_p1'] / df_medium_pagefocus_pivot['CTC_count_all']
df_medium_pagefocus_pivot['prop_CTC_p2']= df_medium_pagefocus_pivot['CTC_count_p2'] / df_medium_pagefocus_pivot['CTC_count_all']
df_medium_pagefocus_pivot['prop_CTC_p3']= df_medium_pagefocus_pivot['CTC_count_p3'] / df_medium_pagefocus_pivot['CTC_count_all']
df_medium_pagefocus_pivot['prop_CTC_p4']= df_medium_pagefocus_pivot['CTC_count_p4'] / df_medium_pagefocus_pivot['CTC_count_all']
df_medium_pagefocus_pivot['prop_CTC_p5']= df_medium_pagefocus_pivot['CTC_count_p5'] / df_medium_pagefocus_pivot['CTC_count_all']
df_medium_pagefocus_pivot['prop_CTC_p6']= df_medium_pagefocus_pivot['CTC_count_p6'] / df_medium_pagefocus_pivot['CTC_count_all']

**Méga jointure**

In [ ]:
df_medium_metrics = pd.merge(left = df_sessions_medium, right = df_ctc_medium, on = 'medium').drop_duplicates()\
                        .merge(df_medium_redirect, on='medium')\
                        .merge(df_medium_pagefocus_pivot, on='medium').drop(columns=['CTC_count_all', 'cumul_time_all'])

**Allocation des coûts**

In [ ]:
# On n'a les coûts que par campagne. On les veut aussi par medium
# On fait une allocation de coûts au prorata du nombre d'événements = par nombre de clics
# On compte le nombre de clics par campagne par medium
df_cost_alloc = df_events[['campaign_id', 'medium']].value_counts()\
                    .to_frame().reset_index().sort_values(['campaign_id', 'medium'])

# On merge avec les données de coûts de la table campaigns et le décompte total des clics par campagne
df_cost_alloc = df_cost_alloc.merge(df_campaigns)\
                    .merge(df_events['campaign_id'].value_counts().to_frame().reset_index(), on="campaign_id")

# On renomme les colonnes
df_cost_alloc = df_cost_alloc.rename(columns = {'count_x':'count_medium', 'count_y':'count_campaign'})

# On procède à l'allocation
df_cost_alloc['cost_alloc'] = round(df_cost_alloc['total_cost'] * df_cost_alloc['count_medium'] / df_cost_alloc['count_campaign'],2)

# On additionne les contributions de chaque campagne aux coûts des 4 media
df_cost_alloc = df_cost_alloc[['medium', 'cost_alloc']].groupby('medium').agg({'cost_alloc': 'sum'}).reset_index()

In [ ]:
# On ajoute ces données de coût à la table
df_medium_metrics = df_medium_metrics.merge(df_cost_alloc, on='medium')

# On calcule les ratios de coûts
df_medium_metrics['cost_per_session'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['session_count']

df_medium_metrics['cost_per_click'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['sum_click']

df_medium_metrics['cost_per_min'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['sum_duration_in_min']

df_medium_metrics['cost_per_ctc'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['sum_ctc']

df_medium_metrics['cost_per_prospect'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['prospect_created']

df_medium_metrics['cost_per_client'] = df_medium_metrics['cost_alloc'] / df_medium_metrics['client_acquired']

**Table finale**

In [ ]:
df_medium_metrics

### 5.A Analyse par partenaire

**Pour chaque partenaire (referrer), on commence par calculer le nombre de clics, le nombre de sessions, le temps en ligne**

In [ ]:
# On extrait les colonnes qui nous intéressent
df_sessions_referrer = df_analysis[['referrer_summary', 'session_id', 'session_duration', 'is_first_session', 'click', 'CTC']]

# On ne garde qu'une ligne par session, puisqu'on a déjà des colonnes qui comptent les clicks / CTC / durée de session
df_sessions_referrer = df_sessions_referrer.drop_duplicates(subset='session_id')

# On remplace les NaN par zéro
df_sessions_referrer[['click', 'CTC']] = df_sessions_referrer[['click', 'CTC']].fillna(0)

# On procède à un groupby + agg
dico_agg = {
    'session_id':'count',
    'session_duration':['sum', 'mean'],
    'is_first_session':['sum', 'mean'],
    'click':['sum', 'mean'],
    'CTC':['sum', 'mean']
}

df_sessions_referrer = df_sessions_referrer.groupby('referrer_summary')\
                            .agg(dico_agg)

df_sessions_referrer = df_sessions_referrer.reset_index()

# On traite chaque bloc du tableau pour supprimer le niveau 0 et renommer les colonnes de la bonne façon
df_sessions_count = df_sessions_referrer[['referrer_summary', 'session_id']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'referrer', 'count':'session_count'})

df_sessions_duration = df_sessions_referrer[['referrer_summary', 'session_duration']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'referrer', 'sum':'sum_duration_in_min', 'mean':'mean_duration_in_min'})

df_sessions_first = df_sessions_referrer[['referrer_summary', 'is_first_session']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'referrer', 'sum':'sum_first_session', 'mean':'prop_first_session'})

df_sessions_click = df_sessions_referrer[['referrer_summary', 'click']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'referrer', 'sum':'sum_click', 'mean':'mean_click'})

df_sessions_ctc = df_sessions_referrer[['referrer_summary', 'CTC']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'referrer', 'sum':'sum_ctc', 'mean':'prop_ctc'})

# On fusionne tout
df_sessions_referrer = df_sessions_count\
                            .merge(df_sessions_duration, on='referrer')\
                            .merge(df_sessions_first, on='referrer')\
                            .merge(df_sessions_click, on='referrer')\
                            .merge(df_sessions_ctc, on='referrer')\

# On arrondit la durée totale en minutes
df_sessions_referrer['sum_duration_in_min'] = round(\
                                                df_sessions_referrer['sum_duration_in_min'].dt.days*24*60 +\
                                                df_sessions_referrer['sum_duration_in_min'].dt.seconds/60\
                                                    ).astype(int)
df_sessions_referrer['mean_duration_in_min'] = round(\
                                                    df_sessions_referrer['mean_duration_in_min'].dt.seconds/60\
                                                        ).astype(int)

**On reprend la table des 1ers CTCs, et on groupe par referrer, pour calculer leurs nombres de prospects créés CTC puis convertis en clients**

In [ ]:
dico_agg = {
    'user_id':'count',
    'prospect_creation_date':'count',
    'is_client':'sum'
}

dico_rename = {
    'referrer_summary':'referrer',
    'user_id':'users_on_CTC',
    'prospect_creation_date':'prospect_created',
    'is_client':'client_acquired'
}

df_ctc_referrer = df_firstctc[['referrer_summary', 'user_id', 'prospect_creation_date', 'is_client']]\
                        .groupby('referrer_summary')\
                        .agg(dico_agg)\
                        .reset_index()\
                        .rename(columns = dico_rename)

In [ ]:
df_ctc_referrer['prospect_created'].sum() + ((df_firstctc['referrer_summary'].isna()) & (df_firstctc['is_client'].notna())).sum()

**Pour chaque referrer, on compte les redirections vers chaque page (au début des sessions)**

In [ ]:
# On commence par ne prendre les événements de df_analysis que lorsque 'referrer' n'est pas le site Garanteo lui-même
# On ne prend que les deux colonnes dont on a besoin
df_referrer_redirect = df_analysis[df_analysis['referrer_summary'].replace('site', np.nan).notna()][['referrer_summary','page']]

# On fait un value_counts qu'on pivote ensuite pour avoir 6 colonnes qui comptent le nombre de redirections sur les 6 pages
df_referrer_redirect = df_referrer_redirect.value_counts().to_frame()\
                            .sort_values(['referrer_summary', 'page']).reset_index()\
                            .pivot(index='referrer_summary', columns='page', values='count').reset_index()\
                            .rename(columns={
                            'referrer_summary':'referrer',
                            '1':'redirect_p1',
                            '2':'redirect_p2',
                            '3':'redirect_p3',
                            '4':'redirect_p4',
                            '5':'redirect_p5',
                            '6':'redirect_p6'
                        })

**Calcul du temps par page et du nombre de CTC sur chaque page, en absolu et en relatif pour chaque medium**

In [ ]:
# Tableau pour afficher des graphes par page en utilisant le paramètre "hue"

# On extrait les colonnes requises du tableau df_sessions_breakdown
df_referrer_pagefocus = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']] \
        
# groupby: on groupe par session et par page
# agg : Pour chaque session, on calcule la somme du temps passé par page, et le nombre de CTC par page 
df_referrer_pagefocus = df_referrer_pagefocus.groupby(['user_id', 'session_started_at', 'page']) \
                            .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'}) \
                            .reset_index() \

# Pour chacune de ces sessions, on ajoute le referrer qui a initié la session. 
# On se débarasse des 'site' qui correspondent à la navigation interne au site de Garanteo pendant la session
df_referrer_pagefocus = df_referrer_pagefocus.merge(df_analysis[['user_id', 'session_started_at', 'referrer_summary']]\
                            .replace('site', np.nan).dropna(subset='referrer_summary'),\
                            on=['user_id', 'session_started_at'])\

# drop: on se débarasse des références à la session 
# groupby : on groupe une nouvelle fois par referrer et par page
# agg : pour chacun des referrers, on calcule le temps passé par les utilisateurs sur chaque page, et le nb de CTC par page 
df_referrer_pagefocus = df_referrer_pagefocus.drop(columns=['user_id', 'session_started_at'])\
                            .groupby(['referrer_summary', 'page'])\
                            .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                            .reset_index()\
                            .rename(columns={
                                    'referrer_summary':'referrer',
                                    'event_duration_in_sec':'cumul_time_in_sec',
                                    'is_CTC':'CTC_count'
                                })

In [ ]:
# Tableau pivoté
# Même principe au début
df_referrer_pagefocus_pivot = df_sessions_breakdown[['user_id', 'session_started_at', 'page', 'event_duration_in_sec', 'is_CTC']]\
                                .groupby(['user_id', 'session_started_at', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()\
                                .merge(df_analysis[['user_id', 'session_started_at', 'referrer_summary']]\
                                       .replace('site', np.nan).dropna(subset='referrer_summary'),\
                                       on=['user_id', 'session_started_at'])\
                                .drop(columns=['user_id', 'session_started_at'])\
                                .groupby(['referrer_summary', 'page'])\
                                .agg({'event_duration_in_sec':'sum', 'is_CTC': 'sum'})\
                                .reset_index()

# On rajoute un pivot pour avoir une colonne par page, pour chaque indicateur
df_referrer_pagefocus_pivot = df_referrer_pagefocus_pivot.pivot(index='referrer_summary', columns='page', values=['event_duration_in_sec', 'is_CTC'])\
                                .reset_index()

# On clean chaque bloc
df_referrer_pagetime = df_referrer_pagefocus_pivot['event_duration_in_sec']\
                        .rename(columns={
                            '1':'cumul_time_p1',
                            '2':'cumul_time_p2',
                            '3':'cumul_time_p3',
                            '4':'cumul_time_p4',
                            '5':'cumul_time_p5',
                            '6':'cumul_time_p6'
                        })

df_referrer_pagectc = df_referrer_pagefocus_pivot['is_CTC']\
                        .rename(columns={
                            '1':'CTC_count_p1',
                            '2':'CTC_count_p2',
                            '3':'CTC_count_p3',
                            '4':'CTC_count_p4',
                            '5':'CTC_count_p5',
                            '6':'CTC_count_p6'
                        })

# On concatène le tout
df_referrer_pagefocus_pivot = pd.concat([
                                df_referrer_pagefocus_pivot['referrer_summary'], 
                                df_referrer_pagetime, 
                                df_referrer_pagectc
                                ],axis=1)\
                            .rename(columns={'referrer_summary':'referrer'})

In [ ]:
# On calcule des totaux pour ensuite calculer les %
df_referrer_pagefocus_pivot['cumul_time_all']=\
    df_referrer_pagefocus_pivot[['cumul_time_p1','cumul_time_p2','cumul_time_p3','cumul_time_p4','cumul_time_p5','cumul_time_p6']]\
    .sum(axis=1)

df_referrer_pagefocus_pivot['CTC_count_all']=\
    df_referrer_pagefocus_pivot[['CTC_count_p1','CTC_count_p2','CTC_count_p3','CTC_count_p4','CTC_count_p5','CTC_count_p6']]\
    .sum(axis=1)

In [ ]:
# Calcul des %
df_referrer_pagefocus_pivot['prop_time_p1']= df_referrer_pagefocus_pivot['cumul_time_p1'] / df_referrer_pagefocus_pivot['cumul_time_all']
df_referrer_pagefocus_pivot['prop_time_p2']= df_referrer_pagefocus_pivot['cumul_time_p2'] / df_referrer_pagefocus_pivot['cumul_time_all']
df_referrer_pagefocus_pivot['prop_time_p3']= df_referrer_pagefocus_pivot['cumul_time_p3'] / df_referrer_pagefocus_pivot['cumul_time_all']
df_referrer_pagefocus_pivot['prop_time_p4']= df_referrer_pagefocus_pivot['cumul_time_p4'] / df_referrer_pagefocus_pivot['cumul_time_all']
df_referrer_pagefocus_pivot['prop_time_p5']= df_referrer_pagefocus_pivot['cumul_time_p5'] / df_referrer_pagefocus_pivot['cumul_time_all']
df_referrer_pagefocus_pivot['prop_time_p6']= df_referrer_pagefocus_pivot['cumul_time_p6'] / df_referrer_pagefocus_pivot['cumul_time_all']

df_referrer_pagefocus_pivot['prop_CTC_p1']= df_referrer_pagefocus_pivot['CTC_count_p1'] / df_referrer_pagefocus_pivot['CTC_count_all']
df_referrer_pagefocus_pivot['prop_CTC_p2']= df_referrer_pagefocus_pivot['CTC_count_p2'] / df_referrer_pagefocus_pivot['CTC_count_all']
df_referrer_pagefocus_pivot['prop_CTC_p3']= df_referrer_pagefocus_pivot['CTC_count_p3'] / df_referrer_pagefocus_pivot['CTC_count_all']
df_referrer_pagefocus_pivot['prop_CTC_p4']= df_referrer_pagefocus_pivot['CTC_count_p4'] / df_referrer_pagefocus_pivot['CTC_count_all']
df_referrer_pagefocus_pivot['prop_CTC_p5']= df_referrer_pagefocus_pivot['CTC_count_p5'] / df_referrer_pagefocus_pivot['CTC_count_all']
df_referrer_pagefocus_pivot['prop_CTC_p6']= df_referrer_pagefocus_pivot['CTC_count_p6'] / df_referrer_pagefocus_pivot['CTC_count_all']

**Méga jointure**

In [ ]:
df_referrer_metrics = pd.merge(left = df_sessions_referrer, right = df_ctc_referrer, on = 'referrer').drop_duplicates()\
                        .merge(df_referrer_redirect, on='referrer')\
                        .merge(df_referrer_pagefocus_pivot, on='referrer')\
                        .drop(columns=['CTC_count_all', 'cumul_time_all'])

**Table finale**

In [ ]:
df_referrer_metrics

## 5. Objectif B

On va repartir de la df_analysis (jointure de sessions et events) créée précédemment

**Données du setup utilisateur**

On vérifie que chaque utilisateur n'a qu'un seul setup (ce qui paraitrait logique, car il y a probablement un user_id par ip) 

In [ ]:
df_analysis[['user_id', 'device_type', 'device_browser', 'device_operating_system', 'device_language', 'country']]\
                        .value_counts().to_frame().reset_index()\
                        ['user_id'].value_counts()

C'est vérifié, sinon on aurait des user_id qui apparaitraient plusieurs fois dans le tableau ci-dessus

Ainsi, il suffira de joindre le setup à la fin, en utilisant comme clé les user_id

In [ ]:
df_setup = df_sessions[['user_id', 'device_type', 'device_browser', 'device_operating_system', 'device_language', 'country']]\
            .drop_duplicates(subset='user_id').reset_index(drop=True)

**Campagne / canal / partenaire de la 1ere visite sur le site**

In [ ]:
# On sélectionne les colonnes qui nous intéressent
df_marketing = df_events[['user_id', 'event_timestamp', 'campaign_id', 'medium', 'referrer_summary']]\

# On trie les événements de chaque user par date / heure d'événement, et on ne garde que le tout premier
df_marketing = df_marketing.sort_values(['user_id', 'event_timestamp'])\
                    .drop_duplicates('user_id', keep='first')\
                    .drop(columns='event_timestamp')\
                    .reset_index(drop=True)

# On renomme les colonnes pour qu'elles soient plus explicites
df_marketing = df_marketing.rename(columns={
    'campaign_id':'first_visit_campaign_id',
    'medium':'first_visit_medium',
    'referrer_summary':'first_visit_referrer'
})

**Nombre de sessions, temps moyen entre chaque session, temps total sur le site, durée moyenne d'une session**

In [ ]:
# On extrait les colonnes qui nous intéressent et on se focalise pour l'instant sur les sessions
# On fait donc un drop_duplicates() car df_analysis contient plusieurs événements par session
df_sessions_users = df_analysis[['user_id', 'session_started_at', 'time_btw_sessions', 'session_duration']]\
                            .drop_duplicates(subset=['user_id', 'session_started_at'])

# On procède à un groupby + agg 
dico_agg = {
    'session_started_at':'count',
    'time_btw_sessions':'mean',
    'session_duration':['sum', 'mean']
}

df_sessions_users = df_sessions_users.groupby('user_id').agg(dico_agg).reset_index()

# On clean chaque bloc
df_sessions_count = df_sessions_users[['user_id', 'session_started_at']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'user_id', 'count':'session_count'})

df_sessions_timebtw = df_sessions_users[['user_id', 'time_btw_sessions']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'user_id', 'sum':'sum_timebtw_in_min', 'mean':'avg_days_btw'})

df_sessions_duration = df_sessions_users[['user_id', 'session_duration']]\
                            .droplevel(level=0, axis=1)\
                            .rename(columns={'':'user_id', 'sum':'sum_duration_in_min', 'mean':'avg_duration_in_min'})

# On fusionne le tout
df_sessions_users = df_sessions_count\
                            .merge(df_sessions_timebtw, on='user_id')\
                            .merge(df_sessions_duration, on='user_id')\

# On convertit les durées
# En minutes :
df_sessions_users['sum_duration_in_min'] = round(\
                                                df_sessions_users['sum_duration_in_min'].dt.days*24*60 +\
                                                df_sessions_users['sum_duration_in_min'].dt.seconds/60\
                                                    )
# En minutes :
df_sessions_users['avg_duration_in_min'] = round(df_sessions_users['avg_duration_in_min'].dt.seconds/60)

# En jours :
df_sessions_users['avg_days_btw'] = df_sessions_users['avg_days_btw'].dt.days

# On remplace les NaN par des 0
df_sessions_users = df_sessions_users.fillna(0)

**Nombre de visites sur chaque page / temps total sur chaque page / % de temps sur chaque page**

In [ ]:
dico_agg = {
    'event_duration_in_sec':['count', 'sum'],
    'is_CTC':'sum'
}

dico_rename_page = {
    'count':'count_visit',
    'sum':'sum_time'
}

dico_rename_ctc = {
    'sum':'ctc_count'
}

# On repart de df_page_visits créé plus haut, dans l'Objectif A 
# On procède à un groupby + agg

df_page_visits = df_page_visits[['user_id', 'page', 'event_duration_in_sec', 'is_CTC']]\
                                .groupby(['user_id', 'page'])\
                                .agg(dico_agg)\
                                .reset_index()

# On clean chaque bloc
df_page = df_page_visits[['event_duration_in_sec']]\
                    .droplevel(level=0, axis=1)\
                    .rename(columns=dico_rename_page)\
                    .fillna(0)

df_page_ctc = df_page_visits[['is_CTC']]\
                    .droplevel(level=0, axis=1)\
                    .rename(columns=dico_rename_ctc)\
                    .fillna(0)

# On concatène le tout
df_page_visits = pd.concat([df_page_visits['user_id'].to_frame(), df_page_visits['page'].to_frame(), df_page, df_page_ctc], axis=1)

In [ ]:
# On pivote chaque bloc pour avoir une colonne par page, par indicateur

dico_rename_count = {
    '1':'visits_p1',
    '2':'visits_p2',
    '3':'visits_p3',
    '4':'visits_p4',
    '5':'visits_p5',
    '6':'visits_p6',
}
dico_rename_sum = {
    '1':'time_in_sec_p1',
    '2':'time_in_sec_p2',
    '3':'time_in_sec_p3',
    '4':'time_in_sec_p4',
    '5':'time_in_sec_p5',
    '6':'time_in_sec_p6',
}

dico_rename_ctc = {
    '1':'ctc_p1',
    '2':'ctc_p2',
    '3':'ctc_p3',
    '4':'ctc_p4',
    '5':'ctc_p5',
    '6':'ctc_p6',
}

df_page_count = df_page_visits[['user_id', 'page','count_visit']]\
                    .pivot(columns='page', index='user_id', values='count_visit')\
                    .reset_index()\
                    .rename(columns=dico_rename_count)\
                    .fillna(0)

df_page_time = df_page_visits[['user_id', 'page', 'sum_time']]\
                    .pivot(columns='page', index='user_id', values='sum_time')\
                    .reset_index(drop=True)\
                    .rename(columns=dico_rename_sum)\
                    .fillna(0)

df_page_ctc = df_page_visits[['user_id', 'page', 'ctc_count']]\
                    .pivot(columns='page', index='user_id', values='ctc_count')\
                    .reset_index(drop=True)\
                    .rename(columns=dico_rename_ctc)\
                    .fillna(0)

# On concatène le tout
df_page_visits = pd.concat([df_page_count, df_page_time, df_page_ctc], axis=1).reset_index(drop=True)

In [ ]:
# On calcule le % de temps passé sur chaque page
sum_time = df_page_visits[['time_in_sec_p1','time_in_sec_p2','time_in_sec_p3','time_in_sec_p4','time_in_sec_p5','time_in_sec_p6']].sum(axis=1)
df_page_visits['perc_time_p1'] = (df_page_visits['time_in_sec_p1'] / sum_time).fillna(0)
df_page_visits['perc_time_p2'] = (df_page_visits['time_in_sec_p2'] / sum_time).fillna(0)
df_page_visits['perc_time_p3'] = (df_page_visits['time_in_sec_p3'] / sum_time).fillna(0)
df_page_visits['perc_time_p4'] = (df_page_visits['time_in_sec_p4'] / sum_time).fillna(0)
df_page_visits['perc_time_p5'] = (df_page_visits['time_in_sec_p5'] / sum_time).fillna(0)
df_page_visits['perc_time_p6'] = (df_page_visits['time_in_sec_p6'] / sum_time).fillna(0)

**Temps de visite par jour de la semaine**

In [ ]:
# On extrait les colonnes qui nous intéressent et on se place à l'échelle des sessions
# df_analysis est à l'échelle des événements, donc on supprime les doublons 
df_weekday = df_analysis[['user_id', 'session_started_at', 'session_weekday', 'session_duration']]\
                    .drop_duplicates(subset=['user_id', 'session_started_at'])

# On supprime la colonne session_started_at pour permettre le groupby
df_weekday = df_weekday.drop(columns='session_started_at')

# On procède au groupby + agg
df_weekday = df_weekday.groupby(['user_id','session_weekday']).agg({'session_duration':['sum','mean']})

# On clean le df
dico_rename = {
    "sum":"total_sessions_duration",
    "mean":"avg_sessions_duration"
}
df_weekday = df_weekday.droplevel(0, axis=1).reset_index().rename(columns=dico_rename)

# On convertit les durées en minutes
df_weekday['total_sessions_duration'] = round(df_weekday['total_sessions_duration'].dt.total_seconds()/60,0)
df_weekday['avg_sessions_duration'] = round(df_weekday['avg_sessions_duration'].dt.total_seconds()/60,0)

# On pivote pour avoir une colonne par jour de la semaine, par indicateur
df_weekday = df_weekday.pivot(
    columns='session_weekday', 
    index='user_id', 
    values=['total_sessions_duration', 'avg_sessions_duration'])

# On clean chaque bloc
dico_rename = {
    0:"minutes_mon",
    1:"minutes_tue",
    2:"minutes_wed",
    3:"minutes_thu",
    4:"minutes_fri",
    5:"minutes_sat",
    6:"minutes_sun",
}

df_weekday_total = df_weekday['total_sessions_duration'].rename(columns=dico_rename).reset_index().fillna(0)

dico_rename = {
    0:"avg_mon",
    1:"avg_tue",
    2:"avg_wed",
    3:"avg_thu",
    4:"avg_fri",
    5:"avg_sat",
    6:"avg_sun",
}

df_weekday_avg = df_weekday['avg_sessions_duration'].rename(columns=dico_rename).reset_index().fillna(0)

**Temps en ligne par moment de la journée**

In [ ]:
# Cf. cellule juste au-dessus. Même code avec, à la place, les moments de la journée

df_period = df_analysis[['user_id', 'session_started_at', 'day_period', 'session_duration']]\
                    .drop_duplicates(subset=['user_id', 'session_started_at'])\
                    .drop(columns='session_started_at')\
                    .groupby(['user_id','day_period'])\
                    .agg({'session_duration':['sum','mean']})
dico_rename = {
    "sum":"total_sessions_duration",
    "mean":"avg_sessions_duration"
}
df_period = df_period.droplevel(0, axis=1).reset_index().rename(columns=dico_rename)
df_period['total_sessions_duration'] = round(df_period['total_sessions_duration'].dt.total_seconds()/60,0)
df_period['avg_sessions_duration'] = round(df_period['avg_sessions_duration'].dt.total_seconds()/60,0)

df_period = df_period.pivot(
    columns='day_period', 
    index='user_id', 
    values=['total_sessions_duration', 'avg_sessions_duration'])

dico_rename = {
    'night':"minutes_night",
    'morning':"minutes_morning",
    'afternoon':"minutes_afternoon",
    'evening':"minutes_evening"
}

df_period_total = df_period['total_sessions_duration'].rename(columns=dico_rename).reset_index().fillna(0)

dico_rename = {
    'night':"avg_night",
    'morning':"avg_morning",
    'afternoon':"avg_afternoon",
    'evening':"avg_evening"
}

df_period_avg = df_period['avg_sessions_duration'].rename(columns=dico_rename).reset_index().fillna(0)

**Méga jointure**

In [ ]:
df_users_metrics = df_sessions_users\
                        .merge(df_page_visits, how='left', on='user_id')\
                        .merge(df_weekday_total, how='left', on='user_id')\
                        .merge(df_weekday_avg, how='left', on='user_id')\
                        .merge(df_period_total, how='left', on='user_id')\
                        .merge(df_period_avg, how='left', on='user_id')\
                        .merge(df_setup, how='left', on='user_id')\
                        .merge(df_marketing, on='user_id', how='left')

**Table finale**

In [ ]:
df_users_metrics

## 5. Objectif C

**Les prospects sont avant tout des users. Je repars donc de ma table construite pour l'Objectif B, avec juste les colonnes qui m'intéressent pour entraîner le modèle de ML de l'Objectif C**

In [ ]:
df_profiles = df_users_metrics[['user_id','session_count','avg_days_btw','sum_duration_in_min']]\
                .fillna(value={'avg_days_btw':0})

**Je lui joins toutes les infos de df_prospects**

In [ ]:
df_profiles = df_profiles.merge(df_prospects)

**On peut joindre les données de setup**

In [ ]:
# Par défaut on ne les rajoute pas
# df_profiles = df_profiles.merge(df_setup, on='user_id')

**Nombre de clics et de CTC**

On repart sur df_analysis

In [ ]:
# On sélectionne les colonnes qui nous intéressent
df_pevents = df_analysis[['user_id', 'session_started_at', 'session_ended_at', 'session_duration', 'event_timestamp', 'event_type', 'CTC', 'click', 'CTC_timestamp', 'first_session_started_at']]\
                    .sort_values(['user_id', 'session_started_at', 'event_timestamp'])

# On remplit les NaN de event_timestamp (session sans event)
df_pevents['event_timestamp'] = df_pevents['event_timestamp'].fillna(df_pevents['session_started_at'])

# On ne garde que les users qui sont dans la table prospects
df_pevents = df_pevents[df_pevents['user_id'].isin(df_prospects['user_id'])].reset_index(drop=True)

In [ ]:
# Les colonnes 'CTC' et 'click' comptent les événements de ces types pour chaque session
# df_pevents est à l'échelle des événements
# On a donc des doublons qu'il faut supprimer pour se placer à l'échelle des sessions
df_clickcount = df_pevents[['user_id', 'session_started_at', 'CTC', 'click']].drop_duplicates()

# Puis on supprime la colonne session_started_at dont on n'a plus besoin, et on procède au groupby + agg(sum)
df_clickcount = df_clickcount.drop(columns='session_started_at')\
                        .groupby('user_id')\
                        .agg({'CTC':'sum', 'click':'sum'})\
                        .reset_index()

**1er CTC de chaque prospect et campagne associée**

In [ ]:
# On réutilise la table df_firstctc et on la filtre pour ne garder 
# 1. que les prospects
# 2. que les colonnes qui nous intéressent
df_firstctc_prospects = df_firstctc[df_firstctc['user_id'].isin(df_prospects['user_id'])][['user_id', 'CTC_timestamp', 'campaign_id']]\
                            .reset_index(drop=True)\
                            .rename(columns={
                                        'CTC_timestamp':'first_ctc_timestamp',
                                        'campaign_id':'first_ctc_campaign_id'
                                    })

**Nombre de visites avant le CTC**

In [ ]:
# On ajoute à notre table pevents (événements par prospects) la date/heure du 1er CTC de chaque prospect
df_pevents = df_pevents.merge(df_firstctc_prospects, on='user_id')

In [ ]:
# On convertit en date
df_pevents['first_ctc_date'] = df_pevents['first_ctc_timestamp'].dt.date
df_pevents['session_date'] = df_pevents['session_started_at'].dt.date

# On crée une colonne de boolean indiquant si la session de l'événement est antérieure à la session du 1er CTC
df_pevents['session_before_first_ctc'] = (df_pevents['session_date'] < df_pevents['first_ctc_date'])

In [ ]:
df_sessions_CTC1 = df_pevents[['user_id', 'session_before_first_ctc', 'session_started_at','session_duration']]
df_sessions_CTC1[df_sessions_CTC1['session_before_first_ctc']==True]

In [ ]:
# On extrait les colonnes qui nous intéressent pour le calcul
df_sessions_CTC1 = df_pevents[['user_id', 'session_started_at', 'session_before_first_ctc']]

# On filtre les sessions antérieures au 1er CTC et on supprime les doublons (plusieurs évévenements par session)
df_sessions_CTC1 = df_sessions_CTC1[df_sessions_CTC1['session_before_first_ctc']==True]\
                            .drop_duplicates(subset=['user_id', 'session_started_at'])

# Puis on procède au groupby + agg pour compter les sessions avant le 1er CTC
df_sessions_CTC1 = df_sessions_CTC1.groupby('user_id')\
                            .agg({'session_started_at':'count'})\
                            .reset_index()\
                            .rename(columns={'session_started_at':'sessions_before_first_ctc'})

# A ce stade on n'a que les prospects qui ont eu au moins 1 session complète avant leur 1er CTC.
# On veut inclure tous les prospects
df_sessions_CTC1 = pd.merge(
                        left = df_prospects['user_id'],
                        right = df_sessions_CTC1,
                        how = 'left',
                        on = 'user_id'
                    )

# En ensuite on remplace les NaN par 0 (cad 0 session avant le 1er CTC, cad la 1ere session est celle du 1er CTC)
df_sessions_CTC1['sessions_before_first_ctc'] = df_sessions_CTC1['sessions_before_first_ctc'].fillna(0)

**Temps total de session et nombre de clics avant le 1er CTC**

In [ ]:
# On commence par ajouter les données par événement et par page calculées plus haut
df_pevents = df_pevents.merge(df_event_duration, how='left', on=['user_id', 'event_timestamp'])

In [ ]:
# On crée une colonne qui indique si l'événement est antérieur au 1er CTC de l'utilisateur
df_pevents['event_before_first_ctc'] = (df_pevents['event_timestamp'] < df_pevents['first_ctc_timestamp'])

In [ ]:
# On extrait les infos pertinentes
df_time_CTC1 = df_pevents[['user_id', 'event_before_first_ctc', 'event_duration_in_sec']]

# On filtre les événements antérieurs au 1er CTC et on procède au groupby + agg
# count: nombre de clics avant le CTC
# sum: temps en ligne avant le CTC
df_time_CTC1 = df_time_CTC1[df_time_CTC1['event_before_first_ctc']==True]\
                            .groupby('user_id')\
                            .agg({'event_duration_in_sec':['count','sum']})\
                            .droplevel(level=0, axis=1)\
                            .reset_index()\
                            .rename(columns={
                                'count':'clicks_before_first_ctc',
                                'sum':'time_online_before_first_ctc_in_sec'
                            })

**Temps calendaire entre la 1ère visite et le 1er CTC**

In [ ]:
# On convertit la différence en jours
df_pevents['days_to_first_ctc'] = (df_pevents['first_ctc_timestamp'] - df_pevents['first_session_started_at']).dt.days

**Page du 1er et du dernier CTC, date du dernier CTC**

In [ ]:
# On filtre df_events par 'event_type' == 'CTC' et on extrait les colonnes utiles
df_firstctc_page = df_events[df_events['event_type']=='CTC'][['user_id','event_timestamp','page']]\

# On trie puis on supprime les doublons pour ne garder que le 1er CTC, dont on ne conserve que la page (et le user_id)
df_firstctc_page = df_firstctc_page.sort_values(['user_id', 'event_timestamp'])\
                        .drop_duplicates(subset='user_id')\
                        .drop(columns='event_timestamp')\
                        .rename(columns={'page':'first_ctc_page'})\
                        .reset_index(drop=True)

# On fait la même chose pour extraire la page du dernier CTC
df_lastctc_page = df_events[df_events['event_type']=='CTC'][['user_id','event_timestamp','page']]\
                        .sort_values(['user_id', 'event_timestamp'], ascending=[True, False])\
                        .drop_duplicates(subset='user_id')\
                        .rename(columns={'page':'last_ctc_page'})\
                        .reset_index(drop=True)

# On convertit le timestamp en date
df_lastctc_page['last_ctc_date'] = df_lastctc_page['event_timestamp'].dt.date 

# On supprime le timestamp dont on n'a plus besoin
df_lastctc_page = df_lastctc_page.drop(columns='event_timestamp')

**Méga jointure**

In [ ]:
df_prospects_metrics = df_profiles.merge(df_clickcount, on='user_id')\
                            .merge(df_sessions_CTC1, on='user_id', how='left').fillna({'sessions_before_first_CTC':0})\
                            .merge(df_time_CTC1, on='user_id', how='left')\
                            .merge(df_pevents[['user_id','days_to_first_ctc', 'first_ctc_campaign_id']].drop_duplicates(), on='user_id', how='left')\
                            .merge(df_firstctc_page, on='user_id', how='left')\
                            .merge(df_lastctc_page, on='user_id', how='left')

**Table finale**

In [ ]:
df_prospects_metrics

# 6. Analyses / Réponses aux questions

## 6. Objectif A

**On trace ici quelques premiers graphiques, mais l'analyse sera menée sur Power BI**

In [ ]:
x = df_campaigns_metrics['campaign_id']
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

sns.barplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['sum_click'], palette = 'Blues', hue=df_campaigns_metrics['campaign_type'], ax=ax1)
sns.stripplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['cost_per_click'], color='red', ax=ax2)

ax1.set_xlabel('Campaign id')
ax1.set_ylabel('Click count')
ax2.set_ylabel('Cost per click')

ax1.set_xticks(x)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=70)

# On calcule la moyenne du coût par clic (cpc)
mean_cpc = round(df_campaigns_metrics['total_cost'].sum() / df_campaigns_metrics['sum_click'].sum(), 2)

# On calcule l'écart-type du cpc (on revient à la formule mathématique pour bien pondérer par le nb de clics) 
std_cpc = round(np.sqrt(1/df_campaigns_metrics['sum_click'].sum()\
                *(df_campaigns_metrics['sum_click']*(df_campaigns_metrics['cost_per_click'] - mean_cpc)**2).sum()),2)
std_over_mean_cpc = round(std_cpc/mean_cpc, 2)

plt.title(f'Cost per click comparison, mean = {mean_cpc}, std = {std_over_mean_cpc*100}% of mean');

In [ ]:
x = df_campaigns_metrics['campaign_id']
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

sns.barplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['sum_ctc'], palette = 'Blues', hue=df_campaigns_metrics['campaign_type'], ax=ax1)
sns.stripplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['cost_per_ctc'], color='red', ax=ax2)

ax1.set_xlabel('Campaign id')
ax1.set_ylabel('CTC count')
ax2.set_ylabel('Cost per CTC')

ax1.set_xticks(x)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=70)

# On calcule la moyenne du coût par ctc (cpctc)
mean_cpctc = round(df_campaigns_metrics['total_cost'].sum() / df_campaigns_metrics['sum_ctc'].sum(), 2)

# On calcule l'écart-type du cpctc (on revient à la formule mathématique pour bien pondérer par le nb de ctc)
std_cpctc = round(np.sqrt(1/df_campaigns_metrics['sum_ctc'].sum()\
                *(df_campaigns_metrics['sum_ctc']*(df_campaigns_metrics['cost_per_ctc'] - mean_cpctc)**2).sum()),2)

std_over_mean_cpctc = round(std_cpctc/mean_cpctc, 2)

plt.title(f'Cost per CTC comparison, mean = {mean_cpctc}, std = {std_over_mean_cpctc*100}% of mean');

In [ ]:
x = df_campaigns_metrics['campaign_id']
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

sns.barplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['prospect_created'], palette = 'Blues', hue=df_campaigns_metrics['campaign_type'], ax=ax1)
sns.stripplot(df_campaigns_metrics, x=x, y=df_campaigns_metrics['cost_per_prospect'], color='red', ax=ax2)

ax1.set_xlabel('Campaign id')
ax1.set_ylabel('Prospect count')
ax2.set_ylabel('Cost per prospect')

ax1.set_xticks(x)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=70)

# On calcule la moyenne du coût par prospect (cppro)
mean_cppro = round(df_campaigns_metrics['total_cost'].sum() / df_campaigns_metrics['prospect_created'].sum(), 2)
std_cppro = round(np.sqrt(1/df_campaigns_metrics['prospect_created'].sum()\
                *(df_campaigns_metrics['prospect_created']*(df_campaigns_metrics['cost_per_prospect'] - mean_cppro)**2).sum()),2)
std_over_mean_cppro = round(std_cppro/mean_cppro, 2)

plt.title(f'Cost per prospect comparison, mean = {mean_cppro}, std = {std_over_mean_cppro*100}% of mean');

**Quelques éléments par page**

In [ ]:
sns.barplot(data=df_campaign_pagefocus, x='campaign_id', y='cumul_time_in_sec', hue='page')
plt.xticks(rotation=70);

In [ ]:
sns.barplot(data=df_campaign_pagefocus, x='campaign_id', y='CTC_count', hue='page')
plt.xticks(rotation=70);

## 6. Objectif B

### Tentative de clustering sur toutes les données de df_users_metrics

**On importe les librairies de clustering**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics  import silhouette_score
from sklearn.preprocessing import MinMaxScaler

**On sépare les variables numériques des variables catégorielles**

In [ ]:
df_users_metrics_num = df_users_metrics[['session_count', 'avg_days_btw', 'sum_duration_in_min',
                                            'avg_duration_in_min', 'visits_p1', 'visits_p2', 'visits_p3',
                                            'visits_p4', 'visits_p5', 'visits_p6', 'time_in_sec_p1',
                                            'time_in_sec_p2', 'time_in_sec_p3', 'time_in_sec_p4', 'time_in_sec_p5',
                                            'time_in_sec_p6', 'perc_time_p1', 'perc_time_p2', 'perc_time_p3', 
                                            'perc_time_p4', 'perc_time_p5', 'perc_time_p6', 'ctc_p1', 'ctc_p2', 'ctc_p3', 
                                            'ctc_p4', 'ctc_p5', 'ctc_p6', 'minutes_mon', 
                                            'minutes_tue', 'minutes_wed', 'minutes_thu', 'minutes_fri', 
                                            'minutes_sat', 'minutes_sun', 'avg_mon', 'avg_tue', 'avg_wed', 
                                            'avg_thu', 'avg_fri', 'avg_sat', 'avg_sun', 'minutes_afternoon', 'minutes_evening',
                                            'minutes_morning', 'minutes_night', 'avg_afternoon', 'avg_evening', 
                                            'avg_morning', 'avg_night']]

df_users_metrics_cat = df_users_metrics[['device_type', 'device_browser', 
                                            'device_operating_system', 'device_language', 'country']]

**Variables numériques**

*Quel scaler utiliser ? Pour le modèle K-means, les deux sont possibles (source: chatgpt). On étudie la distribution des données*

In [ ]:
# Je fixe mon nombre de colonnes
nb_columns = 5

# Ensuite, en faisant la division euclidienne de mon nb de variables par le nb de colonnes, j'obtiens mon nb de lignes 
nb_rows = (df_users_metrics_num.shape[1]//nb_columns)+((df_users_metrics_num.shape[1]%nb_columns)!=0)*1

# Je crée une grille de graphes
fig, axes = plt.subplots(nb_rows, nb_columns, figsize=(3*nb_columns, 3.5*nb_rows))

# J'exécute une boucle pour afficher des histplots de toutes mes variables
for j in range(nb_rows):
    for i in range(nb_columns):
            try:
                sns.histplot(data=df_users_metrics_num, ax=axes[j][i], x=df_users_metrics_num.iloc[:,nb_columns*j+i]);
            except:
                pass

Difficile de conclure. Certaines données semblent normalement distribuées, d'autres non. Dans le doute, on applique le MinMaxScaler à toutes les variables

**On applique le MinMaxScaler**

In [ ]:
scaler = MinMaxScaler()

df_users_metrics_num_norm = pd.DataFrame(scaler.fit_transform(df_users_metrics_num.fillna(0)), columns=df_users_metrics_num.columns)

In [ ]:
df_users_metrics_num_norm.describe()

**On passe les variables catégorielles au .get_dummies()**

In [ ]:
df_users_metrics_cat_norm = pd.get_dummies(df_users_metrics_cat).fillna(False)

**On regroupe le tout**

In [ ]:
df_users_metrics_norm = pd.concat([df_users_metrics['user_id'].to_frame(), df_users_metrics_num_norm, df_users_metrics_cat_norm], axis=1)
df_users_metrics_norm

**On extrait les noms de colonnes** 

In [ ]:
df_users_metrics_norm.drop(columns='user_id').columns

**On crée un dico géant permettant de gérer les éventuels poids**

In [ ]:
weights_dico = {
    'session_count':['session_count',1], 
    'avg_days_btw':['avg_days_btw',1], 
    'sum_duration_in_min':['sum_duration_in_min',1],
    'avg_duration_in_min':['avg_duration_in_min',1],
    'visits_p1':['visits_p1',1],
    'visits_p2':['visits_p2',1], 
    'visits_p3':['visits_p3',1],
    'visits_p4':['visits_p4',1],
    'visits_p5':['visits_p5',1],
    'visits_p6':['visits_p6',1],
    'time_in_sec_p1':['time_in_sec_p1',1],
    'time_in_sec_p2':['time_in_sec_p2',1],
    'time_in_sec_p3':['time_in_sec_p3',1],
    'time_in_sec_p4':['time_in_sec_p4',1],
    'time_in_sec_p5':['time_in_sec_p5',1],
    'time_in_sec_p6':['time_in_sec_p6',1],
    'perc_time_p1':['perc_time_p1',1],
    'perc_time_p2':['perc_time_p2',1],
    'perc_time_p3':['perc_time_p3',1],
    'perc_time_p4':['perc_time_p4',1],
    'perc_time_p5':['perc_time_p5',1],
    'perc_time_p6':['perc_time_p6',1],
    'ctc_p1': ['ctc_p1', 1],
    'ctc_p2': ['ctc_p2', 1],
    'ctc_p3': ['ctc_p3', 1],
    'ctc_p4': ['ctc_p4', 1],
    'ctc_p5': ['ctc_p5', 1],
    'ctc_p6': ['ctc_p6', 1],
    'minutes_mon':['minutes_mon',1],
    'minutes_tue':['minutes_tue',1],
    'minutes_wed':['minutes_wed',1],
    'minutes_thu':['minutes_thu',1],
    'minutes_fri':['minutes_fri',1],
    'minutes_sat':['minutes_sat',1],
    'minutes_sun':['minutes_sun',1],
    'avg_mon':['avg_mon',1],
    'avg_tue':['avg_tue',1],
    'avg_wed':['avg_wed',1],
    'avg_thu':['avg_thu',1],
    'avg_fri':['avg_fri',1],
    'avg_sat':['avg_sat',1],
    'avg_sun':['avg_sun',1],
    'minutes_afternoon':['minutes_afternoon',1],
    'minutes_evening':['minutes_evening',1],
    'minutes_morning':['minutes_morning',1],
    'minutes_night':['minutes_night',1],
    'avg_afternoon':['avg_afternoon',1],
    'avg_evening':['avg_evening',1],
    'avg_morning':['avg_morning',1],
    'avg_night':['avg_night',1],
    'device_type_desktop':['device_type',1],
    'device_type_mobile':['device_type',1],   
    'device_browser_bing':['device_browser',1],
    'device_browser_google':['device_browser',1],
    'device_browser_yahoo':['device_browser',1],
    'device_operating_system_android':['device_operating_system',1], 
    'device_operating_system_ios':['device_operating_system',1],
    'device_operating_system_windows':['device_operating_system',1],
    'device_language_04':['device_language',1],
    'device_language_24077':['device_language',1],
    'device_language_28501':['device_language',1],
    'device_language_85183':['device_language',1],
    'device_language_861':['device_language',1],
    'device_language_EN':['device_language',1],
    'device_language_FR':['device_language',1],
    'country_DE':['country',1],
    'country_FR':['country',1],
    'country_PT':['country',1],
    'country_SP':['country',1],
    'country_UK':['country',1]
}

In [ ]:
# On transforme ensuite le dico en dataframe
df_weights = pd.DataFrame([tup for tup in weights_dico.items()], columns=['column_name', 'weight'])

# On crée une colonne avec les noms d'origine des colonnes (modifiées par le get_dummies)
df_weights['complete_name'] = df_weights['weight'].apply(lambda x: x[0])

# On crée une colonne avec les poids de chaque variable
df_weights['weight'] = df_weights['weight'].apply(lambda x: x[1])

In [ ]:
# On crée une liste des poids non_nuls
X_weights = list(df_weights[df_weights['weight']!=0]['weight'])

# On crée une liste des noms de colonne dont les poids sont nuls
weights_0 = list(df_weights[df_weights['weight']==0]['column_name'])

# On crée une liste des noms de colonne dont les poids sont nuls (noms d'origine non modifiés par le get_dummies)
weights_0_complete = list(df_weights[df_weights['weight']==0]['complete_name'].drop_duplicates())

# Pour pouvoir étudier les résultats du clustering, on crée un df avec seulement les colonnes dont les poids sont non nuls
df_users_metrics_copy = df_users_metrics.drop(columns=weights_0_complete)

In [ ]:
# On extrait nos variables normalisées dont les poids sont non nuls
X = df_users_metrics_norm.drop(columns=weights_0).drop(columns='user_id')

In [ ]:
'''# On étudie les résultats de k-means avec un nombre variable de clusters
# Je mets l'algo entre guillemets pour ne pas le faire tourner à chaque fois que je fais un Run All Cells

list_k = []
list_inertie = []
list_score = []

for k in range(2,30):
    modelKM = KMeans(n_clusters=k, n_init='auto')
    modelKM.fit(X * X_weights)
    list_k.append(k)
    list_inertie.append(modelKM.inertia_)
    list_score.append(silhouette_score(X * X_weights, modelKM.labels_))

fig, ax1 = plt.subplots()

ax1.plot(list_k, list_score, 'r-')
ax1.set_xlabel('k')
ax1.set_ylabel('Score', color='r')
ax1.tick_params('y', colors='r')

ax2 = ax1.twinx()
ax2.plot(list_k, list_inertie, 'b-')
ax2.set_ylabel('Inertie', color='b')
ax2.tick_params('y', colors='b')

plt.show();''';

**Interprétation** : les données de setup, les données temporelles et les données de préférence de page web semblent peu solubles entre elles (max silhouette score ~0.15, atteint pour une inertie élevée, ce qui indique que le modèle est peu robuste)

**Constat :**
Après divers tests sur des sous-ensembles de la table df_users_metrics, le modèle k-means donne des résultats 
- Satisfaisants sur les données de setup utilisateur (max silhouette score > 0.5) 
- Peu satisfaisants sur les données temporelles et les données de pages web

**Action :**
- On utilise le modèle k-means pour créer des clusters de setups utilisateurs
- On crée notre propre arbre de décision pour classer les utilisateurs en fonction de leurs intérêts / préférences web
- On crée notre propre arbre de décision pour classer les utilisateurs en fonction de leurs moments de connexion privilégiés 

### Clustering sur le setup utilisateur

**On amende les poids pour ne garder que les variables normalisées de setup**

In [ ]:
weights_dico = {
    'session_count':['session_count',0], 
    'avg_days_btw':['avg_days_btw',0], 
    'sum_duration_in_min':['sum_duration_in_min',0],
    'avg_duration_in_min':['avg_duration_in_min',0],
    'visits_p1':['visits_p1',0],
    'visits_p2':['visits_p2',0], 
    'visits_p3':['visits_p3',0],
    'visits_p4':['visits_p4',0],
    'visits_p5':['visits_p5',0],
    'visits_p6':['visits_p6',0],
    'time_in_sec_p1':['time_in_sec_p1',0],
    'time_in_sec_p2':['time_in_sec_p2',0],
    'time_in_sec_p3':['time_in_sec_p3',0],
    'time_in_sec_p4':['time_in_sec_p4',0],
    'time_in_sec_p5':['time_in_sec_p5',0],
    'time_in_sec_p6':['time_in_sec_p6',0],
    'perc_time_p1':['perc_time_p1',0],
    'perc_time_p2':['perc_time_p2',0],
    'perc_time_p3':['perc_time_p3',0],
    'perc_time_p4':['perc_time_p4',0],
    'perc_time_p5':['perc_time_p5',0],
    'perc_time_p6':['perc_time_p6',0],
    'ctc_p1': ['ctc_p1', 0],
    'ctc_p2': ['ctc_p2', 0],
    'ctc_p3': ['ctc_p3', 0],
    'ctc_p4': ['ctc_p4', 0],
    'ctc_p5': ['ctc_p5', 0],
    'ctc_p6': ['ctc_p6', 0],
    'minutes_mon':['minutes_mon',0],
    'minutes_tue':['minutes_tue',0],
    'minutes_wed':['minutes_wed',0],
    'minutes_thu':['minutes_thu',0],
    'minutes_fri':['minutes_fri',0],
    'minutes_sat':['minutes_sat',0],
    'minutes_sun':['minutes_sun',0],
    'avg_mon':['avg_mon',0],
    'avg_tue':['avg_tue',0],
    'avg_wed':['avg_wed',0],
    'avg_thu':['avg_thu',0],
    'avg_fri':['avg_fri',0],
    'avg_sat':['avg_sat',0],
    'avg_sun':['avg_sun',0],
    'minutes_afternoon':['minutes_afternoon',0],
    'minutes_evening':['minutes_evening',0],
    'minutes_morning':['minutes_morning',0],
    'minutes_night':['minutes_night',0],
    'avg_afternoon':['avg_afternoon',0],
    'avg_evening':['avg_evening',0],
    'avg_morning':['avg_morning',0],
    'avg_night':['avg_night',0],
    'device_type_desktop':['device_type',1],
    'device_type_mobile':['device_type',1],   
    'device_browser_bing':['device_browser',1],
    'device_browser_google':['device_browser',1],
    'device_browser_yahoo':['device_browser',1],
    'device_operating_system_android':['device_operating_system',1], 
    'device_operating_system_ios':['device_operating_system',1],
    'device_operating_system_windows':['device_operating_system',1],
    'device_language_04':['device_language',1],
    'device_language_24077':['device_language',1],
    'device_language_28501':['device_language',1],
    'device_language_85183':['device_language',1],
    'device_language_861':['device_language',1],
    'device_language_EN':['device_language',1],
    'device_language_FR':['device_language',1],
    'country_DE':['country',1],
    'country_FR':['country',1],
    'country_PT':['country',1],
    'country_SP':['country',1],
    'country_UK':['country',1]
}

In [ ]:
df_weights = pd.DataFrame([tup for tup in weights_dico.items()], columns=['column_name', 'weight'])
df_weights['complete_name'] = df_weights['weight'].apply(lambda x: x[0])
df_weights['weight'] = df_weights['weight'].apply(lambda x: x[1])

In [ ]:
X_weights = list(df_weights[df_weights['weight']!=0]['weight'])
weights_0 = list(df_weights[df_weights['weight']==0]['column_name'])
weights_0_complete = list(df_weights[df_weights['weight']==0]['complete_name'].drop_duplicates())

df_users_setup = df_users_metrics.drop(columns=weights_0_complete)

In [ ]:
X = df_users_metrics_norm.drop(columns=weights_0).drop(columns='user_id')

In [ ]:
# Je le mets entre crochets pour ne pas le faire tourner à chaque fois que je fais un Run All Cells

'''list_k = []
list_inertie = []
list_score = []

for k in range(2,30):
    modelKM = KMeans(n_clusters=k, n_init='auto')
    modelKM.fit(X * X_weights)
    list_k.append(k)
    list_inertie.append(modelKM.inertia_)
    list_score.append(silhouette_score(X * X_weights, modelKM.labels_))

fig, ax1 = plt.subplots()

ax1.plot(list_k, list_score, 'r-')
ax1.set_xlabel('k')
ax1.set_ylabel('Score', color='r')
ax1.tick_params('y', colors='r')

ax2 = ax1.twinx()
ax2.plot(list_k, list_inertie, 'b-')
ax2.set_ylabel('Inertie', color='b')
ax2.tick_params('y', colors='b')

plt.show();''';

Il semblerait que n = 12 soit un bon compromis ("coude" de la courbe d'inertie, score de silhouette > 0.5)

In [ ]:
n = 12

In [ ]:
modelKM = KMeans(n_clusters=n, n_init='auto')
modelKM.fit(X * X_weights)
df_users_setup['setup_cluster'] = pd.DataFrame(modelKM.labels_)

In [ ]:
# On compte le nb de users dans chaque cluster
df_users_setup['setup_cluster'].value_counts()

In [ ]:
# On regarde un échantillon de chaque cluster pour comprendre sur quelles bases ils sont regroupés
for i in range(n):
    print(f'cluster {i}')
    display(df_users_setup[df_users_setup['setup_cluster']==i].sample(10))

### Arbre de décision pour grouper les users par intérêts / préférences de pages web

**Arbre de décision proposé :**

- **Etape 1** : On regarde si l'utilisateur a fait un ou plusieurs CTC
    - *Cas 1.1* : il a fait au moins un CTC sur plusieurs pages différentes -> cet utilisateur a des "préférences multiples"
    - *Cas 1.2* : il a fait au moins un CTC sur une page unique -> cette page devient sa page préférée
    - *Cas 1.3* : il n'a pas fait de CTC -> on passe à l'étape 2
- **Etape 2** : On regarde le temps (en sec) passé par l'utilisateur sur chaque page. On compte le nombre de pages où il a passé plus de temps que l'utilisateur médian. Le cas échéant, on considère qu'il est intéressé par cette page.
    - *Cas 2.1* : il est intéressé par plusieurs pages
        - Cas 2.1.1 : une de ces pages se démarque des autres en % de temps consacré (eg. si la page sur laquelle il a passé le plus de temps dépasse de 20 ppts la 2e page sur laquelle il a passé le plus de temps) -> cette page devient sa page préférée
        - Cas 2.1.2 : aucune page ne se démarque des autres -> cet utilisateur a des "préférences multiples"
    - *Cas 2.2* : il est intéressé par une page unique -> cette page devient sa page préférée
    - *Cas 2.3* : il n'est intéressé par aucune page -> cet utilisateur n'a qu'un "intérêt limité"

**On crée la table requise**

In [ ]:
# On extrait les données requises
subdf_users_metrics = df_users_metrics[['user_id', 'session_count', 'visits_p1', 'visits_p2', 'visits_p3',
                   'visits_p4', 'visits_p5', 'visits_p6', 'time_in_sec_p1',
                   'time_in_sec_p2', 'time_in_sec_p3', 'time_in_sec_p4', 'time_in_sec_p5',
                   'time_in_sec_p6', 'perc_time_p1', 'perc_time_p2', 'perc_time_p3',
                   'perc_time_p4', 'perc_time_p5', 'perc_time_p6', 'ctc_p1', 'ctc_p2', 'ctc_p3', 'ctc_p4', 'ctc_p5', 'ctc_p6']]

df_page_pref = subdf_users_metrics[['user_id', 'session_count']]

# Pour l'étape 1 : l'utilisateur a-t-il fait un CTC sur les pages ?
df_page_pref['ctc_p1'] = (subdf_users_metrics['ctc_p1'] > 0)
df_page_pref['ctc_p2'] = (subdf_users_metrics['ctc_p2'] > 0)
df_page_pref['ctc_p3'] = (subdf_users_metrics['ctc_p3'] > 0)
df_page_pref['ctc_p4'] = (subdf_users_metrics['ctc_p4'] > 0)
df_page_pref['ctc_p5'] = (subdf_users_metrics['ctc_p5'] > 0)
df_page_pref['ctc_p6'] = (subdf_users_metrics['ctc_p6'] > 0)

# On compte les pages (distinctes) où il a fait un CTC
df_page_pref['ctc_page_count'] = df_page_pref[['ctc_p1', 'ctc_p2', 'ctc_p3', 'ctc_p4', 'ctc_p5', 'ctc_p6']].sum(axis=1)

# Pour l'étape 2 : temps sur chaque page
df_page_pref['time_p1'] = subdf_users_metrics['time_in_sec_p1']
df_page_pref['time_p2'] = subdf_users_metrics['time_in_sec_p2']
df_page_pref['time_p3'] = subdf_users_metrics['time_in_sec_p3']
df_page_pref['time_p4'] = subdf_users_metrics['time_in_sec_p4']
df_page_pref['time_p5'] = subdf_users_metrics['time_in_sec_p5']
df_page_pref['time_p6'] = subdf_users_metrics['time_in_sec_p6']

# Pour l'étape 2 : l'utilisateur a-t-il passé plus de temps sur chaque page que l'utilisateur median ?
df_page_pref['liked_p1'] = (subdf_users_metrics['time_in_sec_p1'] >= subdf_users_metrics['time_in_sec_p1'].median())
df_page_pref['liked_p2'] = (subdf_users_metrics['time_in_sec_p2'] >= subdf_users_metrics['time_in_sec_p2'].median())
df_page_pref['liked_p3'] = (subdf_users_metrics['time_in_sec_p3'] >= subdf_users_metrics['time_in_sec_p3'].median())
df_page_pref['liked_p4'] = (subdf_users_metrics['time_in_sec_p4'] >= subdf_users_metrics['time_in_sec_p4'].median())
df_page_pref['liked_p5'] = (subdf_users_metrics['time_in_sec_p5'] >= subdf_users_metrics['time_in_sec_p5'].median())
df_page_pref['liked_p6'] = (subdf_users_metrics['time_in_sec_p6'] >= subdf_users_metrics['time_in_sec_p6'].median())

# Pour l'étape 2 : on compte les pages où il a passé plus de temps que l'utilisateur median
df_page_pref['page_liked_count'] = df_page_pref[['liked_p1', 'liked_p2', 'liked_p3', 'liked_p4', 'liked_p5', 'liked_p6']].sum(axis=1)

# Pour l'étape 2.1 : on calcule l'écart (en points de pourcentage) entre :
    # la page où il a passé le plus de temps = row.max(), et
    # la 2e page où il a passé le plus de temps = row.nlargest(2).iloc[-1]
    
df_page_pref['perc_gap'] = subdf_users_metrics[['perc_time_p1', 'perc_time_p2', 'perc_time_p3', 'perc_time_p4', 'perc_time_p5', 'perc_time_p6']]\
                                .apply(lambda row: row.max() - row.nlargest(2).iloc[-1], axis=1)

In [ ]:
# On définit la fonction qui reproduit l'arbre de décision expliqué ci-avant
def user_preference(row):
    if row['ctc_page_count'] > 1:
        return 'Multipages'
    elif row['ctc_page_count'] == 1:
        return f"Page {row[['ctc_p1','ctc_p2','ctc_p3','ctc_p4','ctc_p5','ctc_p6']].idxmax()[-1]}"
    else:
        if row['page_liked_count'] > 1:
            if row['perc_gap'] >= 0.2:
                return f"Page {row[['time_p1', 'time_p2', 'time_p3', 'time_p4', 'time_p5', 'time_p6']].idxmax()[-1]}"
            else:
                return 'Multipages'
        elif row['page_liked_count'] == 1:
            return f"Page {row[['liked_p1', 'liked_p2', 'liked_p3', 'liked_p4', 'liked_p5', 'liked_p6']].idxmax()[-1]}"
        else:
            return "Peu intéressé"

In [ ]:
# On crée la colonne de segmentation et on regarde les résultats
df_page_pref['user_preference'] = df_page_pref.apply(user_preference, axis=1)
df_page_pref['user_preference'].value_counts()

### Arbre de décision pour grouper les users par comportement (habitudes de connexion)

**On cherche d'éventuelles corrélations entre les variables temporelles (jour de la semaine / moment de la journée)**

In [ ]:
# On extrait les données temporelles
df_timefocus = df_users_metrics[['user_id','minutes_mon', 'minutes_tue', 'minutes_wed', 'minutes_thu', 'minutes_fri', 'minutes_sat', 'minutes_sun', 
                                 'minutes_afternoon', 'minutes_evening', 'minutes_morning', 'minutes_night']]

# On regroupe par jour de semaine / jour de weekend
df_timefocus['minutes_weekday'] = df_timefocus[['minutes_mon', 'minutes_tue', 'minutes_wed', 'minutes_thu', 'minutes_fri']].sum(axis=1)
df_timefocus['total_minutes_days'] = df_timefocus[['minutes_mon', 'minutes_tue', 'minutes_wed', 'minutes_thu', 'minutes_fri', 'minutes_sat', 'minutes_sun']].sum(axis=1)
df_timefocus['perc_on_weekday'] = df_timefocus['minutes_weekday'] / df_timefocus['total_minutes_days']

# On regroupe par temps de jour / temps de nuit
df_timefocus['minutes_workinghours'] = df_timefocus[['minutes_morning', 'minutes_afternoon']].sum(axis=1)
df_timefocus['total_minutes_period'] = df_timefocus[['minutes_afternoon', 'minutes_evening', 'minutes_night', 'minutes_morning']].sum(axis=1)
df_timefocus['perc_on_workinghours'] = df_timefocus['minutes_workinghours'] / df_timefocus['total_minutes_period']

# On cherche une éventuelle corrélation
df_timefocus[['perc_on_weekday','perc_on_workinghours']].corr()

Aucune corrélation.

Mais on peut quand même créer des segments.

**Proposition d'arbre de décision**

Jour / nuit :
- day (% connexion de jour > 2/3)
- night (% connexion de nuit > 2/3)
- no pattern (autres cas)

Semaine / weekend :
- weekday (% connexion en semaine > 2/3)
- weekend (% connexion le weekend > 2/3)
- no pattern (autres cas)

Ce qui donne :
- (day, weekday) = weekday daytime
- (day, weekend) = weekend daytime
- (day, no pattern) = daytime
- (night, weekday) = weekday nighttime
- (night, weekend) = weekend nighttime
- (night, no pattern) = nighttime
- (no pattern, weekday) = weekday
- (no pattern, weekend) = weekend
- (no pattern, no pattern) = no pattern

In [ ]:
# On définit la fonction qui reproduit l'arbre ci-avant
def user_behavior(row):
    string = ''
    if row['perc_on_weekday'] > 2/3:
        string += 'Semaine'
    elif row['perc_on_weekday'] < 1/3:
        string += 'Weekend'
    else:
        string = ''
    if row['perc_on_workinghours'] > 2/3:
        string += ' Jour'
    elif row['perc_on_workinghours'] < 1/3:
        string += ' Nuit'
    else:
        if string == '':
            string = 'Pas de préférence'
        else:
            string += ''
    return string.strip()

In [ ]:
# On crée la colonne de segmentation et on étudie les résultats
df_timefocus['user_behavior'] = df_timefocus.apply(user_behavior, axis=1)
df_timefocus['user_behavior'].value_counts()

### Conclusion : jointure des différentes segmentations

In [ ]:
df_users_segments = df_users_setup[['user_id', 'setup_cluster']]\
                                .merge(df_page_pref[['user_id', 'user_preference']], on='user_id')\
                                .merge(df_timefocus[['user_id', 'user_behavior']], on='user_id')

**Table finale**

In [ ]:
df_users_segments

In [ ]:
df_users_segments[['setup_cluster', 'user_preference', 'user_behavior']].value_counts().to_frame()\
                        .sort_values(['setup_cluster', 'count'], ascending=[True, False]).reset_index()

## 6. Objectif C

**On sépare les variables numériques des variables catégorielles**

In [ ]:
# On enlève les colonnes dont on ne se servira pas
df_prospects_metrics_vf = df_prospects_metrics\
                            .drop(columns=['prospect_id', 'prospect_creation_date', 'age_cat_w20', 'age_cat_w30'])

df_prospects_num = df_prospects_metrics_vf[['session_count', 'avg_days_btw', 'sum_duration_in_min', 'age', 'CTC', 'click', 'clicks_before_first_ctc', 'sessions_before_first_ctc', 'time_online_before_first_ctc_in_sec', 'days_to_first_ctc']]

df_prospects_cat = df_prospects_metrics_vf[['gender', 'first_ctc_page', 'last_ctc_page']]

**Variables numériques**

*On étudie la distribution des données*

In [ ]:
# Même principe que plus haut

nb_columns = 5
nb_rows = (df_prospects_num.shape[1]//nb_columns)+((df_prospects_num.shape[1]%nb_columns)!=0)*1

fig, axes = plt.subplots(nb_rows, nb_columns, figsize=(3*nb_columns, 3.5*nb_rows))

for j in range(nb_rows):
    for i in range(nb_columns):
            try:
                sns.histplot(data=df_prospects_num, ax=axes[j][i], x=df_prospects_num.iloc[:,nb_columns*j+i]);
            except:
                pass

**On passe les variables numériques au minmaxscaler**

In [ ]:
scaler_minmax = MinMaxScaler()

df_prospects_num_norm = pd.DataFrame(scaler_minmax.fit_transform(df_prospects_num), columns=df_prospects_num.columns)

**On passe les variables catégorielles au get_dummies**

In [ ]:
df_prospects_cat_norm = pd.get_dummies(df_prospects_cat)

**On regroupe le tout**

In [ ]:
df_prospects_Xnorm = pd.concat(axis = 1, objs = [
                                            df_prospects_metrics_vf['user_id'],
                                            df_prospects_metrics_vf['is_presented_prospect'],
                                            df_prospects_metrics_vf['is_client'],
                                            df_prospects_num_norm, 
                                            df_prospects_cat_norm
                                                ])

**On se focalise sur le Groupe 1**

In [ ]:
# Groupe 1 = prospects déjà appelés par les sales
# On extrait les variables d'entrainement
df_group_1_Xnorm = df_prospects_Xnorm[df_prospects_Xnorm['is_presented_prospect']==1]\
                                    .drop(columns=['user_id','is_presented_prospect', 'is_client'])

In [ ]:
# On extrait la variable à prédire
df_group_1_y = df_prospects_metrics_vf[df_prospects_metrics_vf['is_presented_prospect']==1]['is_client']
df_group_1_y

**Choix du modèle**

Après consultation de chatgpt, il semblerait que le modèle le plus adapté soit le modèle LogisticRegression

Lien vers le chat: https://chatgpt.com/c/66f3c82f-5ca8-8013-9e40-1cc4245ddb28

**On importe les librairies**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

**On splitte le Groupe 1 en un groupe d'entraînement et un groupe de test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_group_1_Xnorm, df_group_1_y, train_size = 0.80)

**On entraîne le modèle et on calcule ses scores**

In [ ]:
modelLR = LogisticRegression()
modelLR.fit(X_train, y_train)

In [ ]:
train_score = modelLR.score(X_train, y_train)
test_score = modelLR.score(X_test, y_test)

print(f"Score d'entrainement : {train_score} \n Score de test : {test_score}")

**On se focalise maintenant sur le Groupe 2**

In [ ]:
# Groupe 2 = groupe dont on veut prédire la proba de devenir client s'ils sont appelés
# On étudie donc les prospects pas encore clients ET pas encore appelés
# On extrait les variables
df_group_2_Xnorm = df_prospects_Xnorm[(df_prospects_Xnorm['is_presented_prospect']==0) & (df_prospects_Xnorm['is_client']==0)]\
                        .drop(columns=['user_id', 'is_presented_prospect', 'is_client'])

**On effectue la prédicition et on sort les probas**

In [ ]:
group_2_predict_LR = modelLR.predict_proba(df_group_2_Xnorm)
pd.DataFrame(group_2_predict_LR)[1].describe()

In [ ]:
df_prospects[df_prospects['is_presented_prospect']==1]['is_client'].value_counts(normalize=True)

Les résultats ne sont pas délirants : j'ai 7% de clients dans mon Groupe 1, et 7% de proba moyenne sur mon Groupe 2

**On concatène les résultats**

In [ ]:
df_scoring = df_prospects_metrics[(df_prospects_metrics['is_presented_prospect']==0)\
                                        & (df_prospects_metrics['is_client']==0)]['user_id'].to_frame().reset_index(drop=True)

df_scoring['proba'] = pd.DataFrame(group_2_predict_LR)[1]

**On crée des notes de 1 à 5, sur la base des probas. Les seuils sont uniformément répartis**

In [ ]:
df_scoring['score'] = pd.cut(df_scoring['proba'], bins=5, labels=[1,2,3,4,5])

**Table finale**

In [ ]:
df_scoring

**On étudie le nombre de prospects par catégorie**

In [ ]:
df_scoring['score'].value_counts()

# 7. Extraction des tables pour Power BI

In [ ]:
import os

In [ ]:
'''now = dt.datetime.now()

# On crée un folder à la date & heure de l'enregistrement, pour conserver l'historique
folder_name = dt.datetime.isoformat(now, sep = "-", timespec = "seconds").replace(':','')
os.makedirs(folder_name, exist_ok=True)
df_events.to_csv(f'{folder_name}/events.csv')
df_sessions.to_csv(f'{folder_name}/sessions.csv')
df_campaigns_metrics.to_csv(f'{folder_name}/campaigns_metrics.csv')
df_medium_metrics.to_csv(f'{folder_name}/media_metrics.csv')
df_referrer_metrics.to_csv(f'{folder_name}/referrer_metrics.csv')
df_users_metrics.to_csv(f'{folder_name}/users_metrics.csv')
df_users_segments.to_csv(f'{folder_name}/users_segments.csv')
df_prospects_metrics.to_csv(f'{folder_name}/prospects_metrics.csv')
df_scoring.to_csv(f'{folder_name}/scoring.csv')

# On enregistre les fichiers dans le folder output pour PBI
df_events.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/events.csv')
df_sessions.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/sessions.csv')
df_campaigns_metrics.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/campaigns_metrics.csv')
df_medium_metrics.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/media_metrics.csv')
df_referrer_metrics.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/referrer_metrics.csv')
df_users_metrics.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/users_metrics.csv')
df_users_segments.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/users_segments.csv')
df_prospects_metrics.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/prospects_metrics.csv')
df_scoring.to_csv('C:/Users/clem2/Documents/Databird/5. Projet/output/scoring.csv')''';